The main modules are:

Backbone Network (Feature Extraction)

Region Proposal Network (RPN)

Dense Local Regression Module

Discriminative RoI Pooling

Binary Overlap Prediction

Object Classification

Instance Segmentation Extension (optional for segmentation tasks)

Backbone Network (Feature Extraction)
Description: This block extracts feature maps from the input image using a ResNet backbone with Feature Pyramid Network (FPN).

Input:
input_image (torch.Tensor): Shape [B, 3, H, W], where B is batch size, 3 is the RGB channel, H and W are height and width.

Output:
feature_maps (dict of torch.Tensor): A dictionary containing multiple feature maps with different scales, e.g., {P3: [B, 256, H/8, W/8], P4: [B, 256, H/16, W/16], ...}.

In [201]:
# Core Imports
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.ops import FeaturePyramidNetwork, MultiScaleRoIAlign
from torchvision.transforms import functional as F

# Visualization and Utilities (for testing and debugging)
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

# RPN Imports
from torchvision.ops import nms
import torch.nn.functional as F
from torchvision.ops import nms
import torch
from torchvision.ops import roi_align
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CocoDetection
from torchvision import transforms
from torch.utils.data import DataLoader, Subset
import os
import random
from torch.utils.data import random_split
import numpy as np
import torch  # Add import for torch if it's missing
!pip install -q pycocotools
import torch
import torch.optim as optim
from tqdm import tqdm
import os

# Ensure CUDA is available for faster processing
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

!pip freeze > requirements.txt


Using device: cpu


In [2]:


class CustomFeaturePyramidNetwork(nn.Module):
    def __init__(self):
        super(CustomFeaturePyramidNetwork, self).__init__()

        # Lateral connections to project the feature maps without altering channels
        self.lateral_p3 = nn.Conv2d(256, 256, kernel_size=1)
        self.lateral_p4 = nn.Conv2d(512, 512, kernel_size=1)
        self.lateral_p5 = nn.Conv2d(1024, 1024, kernel_size=1)
        self.lateral_p6 = nn.Conv2d(2048, 2048, kernel_size=1)

        # Top-down pathway adjustments to make the channels match across levels
        self.adjust_p5 = nn.Conv2d(2048, 1024, kernel_size=1)  # Adjust p6 channels to 1024 for adding to p5
        self.adjust_p4 = nn.Conv2d(1024, 512, kernel_size=1)   # Adjust p5 channels to 512 for adding to p4
        self.adjust_p3 = nn.Conv2d(512, 256, kernel_size=1)    # Adjust p4 channels to 256 for adding to p3

    def forward(self, features):
        """
        Args:
            features (dict): Dictionary of feature maps from backbone with different channel depths.

        Returns:
            pyramid (dict): Multi-scale feature pyramid with different channel depths.
        """
        # Apply lateral connections to get the same spatial dimensions
        p3 = self.lateral_p3(features['P3'])
        p4 = self.lateral_p4(features['P4'])
        p5 = self.lateral_p5(features['P5'])
        p6 = self.lateral_p6(features['P6'])

        # Top-down pathway with channel adjustment
        p5 = p5 + F.interpolate(self.adjust_p5(p6), size=p5.shape[-2:], mode="nearest")
        p4 = p4 + F.interpolate(self.adjust_p4(p5), size=p4.shape[-2:], mode="nearest")
        p3 = p3 + F.interpolate(self.adjust_p3(p4), size=p3.shape[-2:], mode="nearest")

        # Return the pyramid with different channels maintained
        pyramid = {'P3': p3, 'P4': p4, 'P5': p5, 'P6': p6}
        return pyramid

        return pyramid

def backbone_network(input_image):
    """
    Extracts multi-scale feature maps from the input image using ResNet with a custom FPN that maintains
    different channel sizes at each pyramid level.

    Args:
        input_image (torch.Tensor): Shape [B, 3, H, W]

    Returns:
        feature_maps (dict): Dictionary containing feature maps at multiple scales with different channels.
    """
    # Load ResNet and place it on the correct device
    resnet = models.resnet101(weights="ResNet101_Weights.IMAGENET1K_V1").to(device)

    # Extract specific layers that match the original FPN's expected inputs
    layer1 = nn.Sequential(*list(resnet.children())[:5])   # Outputs 256 channels
    layer2 = nn.Sequential(*list(resnet.children())[5])    # Outputs 512 channels
    layer3 = nn.Sequential(*list(resnet.children())[6])    # Outputs 1024 channels
    layer4 = nn.Sequential(*list(resnet.children())[7])    # Outputs 2048 channels

    # Move layers to the appropriate device
    layer1, layer2, layer3, layer4 = layer1.to(device), layer2.to(device), layer3.to(device), layer4.to(device)

    # Forward pass through each layer to get feature maps
    with torch.no_grad():
        x = input_image
        features = {}
        x = layer1(x)
        features['P3'] = x  # 256 channels
        x = layer2(x)
        features['P4'] = x  # 512 channels
        x = layer3(x)
        features['P5'] = x  # 1024 channels
        x = layer4(x)
        features['P6'] = x  # 2048 channels

    # Use the custom FPN to create a feature pyramid without altering channel depths
    custom_fpn = CustomFeaturePyramidNetwork().to(device)
    feature_maps = custom_fpn(features)

    return feature_maps


In [ ]:
# Testing the backbone function with shape output
def test_backbone_network():
    # Create a dummy input image tensor [B, 3, H, W] with batch size 1, 3 color channels, 256x256 resolution
    dummy_image = torch.rand(1, 3, 256, 256).to(device)

    # Pass the dummy image through the backbone network
    feature_maps = backbone_network(dummy_image)

    # Print the shape of each feature map
    for level, fmap in feature_maps.items():
        print(f"{level} feature map shape: {fmap.shape}")

# Run the test
test_backbone_network()


P3 feature map shape: torch.Size([1, 256, 64, 64])
P4 feature map shape: torch.Size([1, 512, 32, 32])
P5 feature map shape: torch.Size([1, 1024, 16, 16])
P6 feature map shape: torch.Size([1, 2048, 8, 8])


Anchor Generator: Generates a set of anchors (bounding boxes) for each spatial location on the feature maps, using various sizes and aspect ratios.

Objectness and Box Regression Heads: Predicts scores indicating if an anchor contains an object and provides bounding box adjustments.

Proposal Filter: Selects a subset of high-quality proposals using post-processing techniques like non-maximum suppression (NMS) and score filtering.

In [4]:
class AnchorGenerator:
    def __init__(self, sizes, aspect_ratios):
        """
        Initializes the Anchor Generator.

        Args:
            sizes (list of tuples): Each tuple contains anchor sizes for one feature map level.
            aspect_ratios (list of tuples): Aspect ratios for anchors at each level.
        """
        self.sizes = sizes
        self.aspect_ratios = aspect_ratios

    def generate_anchors(self, feature_map_shapes):
        """
        Generates anchors for each feature map level.

        Args:
            feature_map_shapes (list of tuples): Shapes of feature maps as (height, width).

        Returns:
            anchors (list of torch.Tensor): A list of tensors containing anchors for each feature map level.
        """
        anchors = []
        for size, aspect_ratio, shape in zip(self.sizes, self.aspect_ratios, feature_map_shapes):
            # Generate anchors for this feature map shape
            # Placeholder: Implement anchor generation logic here
            anchors.append(torch.rand(shape[0], shape[1], len(size) * len(aspect_ratio), 4))  # Shape [H, W, num_anchors, 4]
        return anchors


class MultiLevelRPN(nn.Module):
    def __init__(self, num_anchors):
        """
        Initializes the RPN with separate objectness and regression heads for each feature map level.

        Args:
            num_anchors (int): Number of anchors per spatial location.
        """
        super(MultiLevelRPN, self).__init__()

        # Define separate RPN heads for each FPN level
        self.shared_convs = nn.ModuleDict({
            "P3": nn.Conv2d(256, 512, kernel_size=3, padding=1),
            "P4": nn.Conv2d(512, 512, kernel_size=3, padding=1),
            "P5": nn.Conv2d(1024, 512, kernel_size=3, padding=1),
            "P6": nn.Conv2d(2048, 512, kernel_size=3, padding=1)
        })

        # Define objectness and regression heads
        self.objectness_heads = nn.ModuleDict({
            "P3": nn.Conv2d(512, num_anchors, kernel_size=1),
            "P4": nn.Conv2d(512, num_anchors, kernel_size=1),
            "P5": nn.Conv2d(512, num_anchors, kernel_size=1),
            "P6": nn.Conv2d(512, num_anchors, kernel_size=1)
        })
        self.box_regression_heads = nn.ModuleDict({
            "P3": nn.Conv2d(512, num_anchors * 4, kernel_size=1),
            "P4": nn.Conv2d(512, num_anchors * 4, kernel_size=1),
            "P5": nn.Conv2d(512, num_anchors * 4, kernel_size=1),
            "P6": nn.Conv2d(512, num_anchors * 4, kernel_size=1)
        })

    def forward(self, feature_maps):
        """
        Forward pass through the RPN.

        Args:
            feature_maps (dict): Dictionary of feature maps from FPN levels with varying channel depths.

        Returns:
            objectness (list of torch.Tensor): Objectness scores for each anchor.
            box_regression (list of torch.Tensor): Box adjustments for each anchor.
        """
        objectness, box_regression = [], []

        for level, fmap in feature_maps.items():
            # Shared conv layer specific to each level
            t = F.relu(self.shared_convs[level](fmap))  # Output shape: [B, 512, H, W]

            # Objectness and regression heads for each level
            obj = self.objectness_heads[level](t)       # Shape: [B, num_anchors, H, W]
            reg = self.box_regression_heads[level](t)   # Shape: [B, num_anchors * 4, H, W]

            objectness.append(obj)
            box_regression.append(reg)

        return objectness, box_regression


In [ ]:
def test_rpn():
    # Dummy feature maps from 4 FPN levels
    feature_maps = {
        "P3": torch.rand(1, 256, 64, 64).to(device),
        "P4": torch.rand(1, 512, 32, 32).to(device),
        "P5": torch.rand(1, 1024, 16, 16).to(device),
        "P6": torch.rand(1, 2048, 8, 8).to(device)
    }

    # Instantiate the RPN
    num_anchors = 9  # Assume 3 scales and 3 aspect ratios per level
    rpn = MultiLevelRPN(num_anchors=num_anchors).to(device)

    # Run the RPN and get objectness and box regression outputs
    objectness, box_regression = rpn(feature_maps)

    # Print the shape of each output for verification
    for i, level in enumerate(feature_maps.keys()):
        print(f"{level} objectness shape: {objectness[i].shape}")
        print(f"{level} box regression shape: {box_regression[i].shape}")

# Run the test
test_rpn()


P3 objectness shape: torch.Size([1, 9, 64, 64])
P3 box regression shape: torch.Size([1, 36, 64, 64])
P4 objectness shape: torch.Size([1, 9, 32, 32])
P4 box regression shape: torch.Size([1, 36, 32, 32])
P5 objectness shape: torch.Size([1, 9, 16, 16])
P5 box regression shape: torch.Size([1, 36, 16, 16])
P6 objectness shape: torch.Size([1, 9, 8, 8])
P6 box regression shape: torch.Size([1, 36, 8, 8])


Filter Proposals

In [121]:



def filter_proposals(objectness, box_regression, anchors, image_size, pre_nms_top_n=1000, post_nms_top_n=300, nms_thresh=0.7):
    """
    Filters proposals using objectness scores, box regression, and NMS.

    Args:
        objectness (list of torch.Tensor): List of objectness scores for each anchor, one per FPN level.
        box_regression (list of torch.Tensor): List of box adjustments for each anchor, one per FPN level.
        anchors (list of torch.Tensor): List of anchors for each FPN level.
        image_size (tuple): Tuple of (height, width) of the input image.
        pre_nms_top_n (int): Number of top proposals to keep before NMS.
        post_nms_top_n (int): Number of top proposals to keep after NMS.
        nms_thresh (float): Threshold for non-maximum suppression.

    Returns:
        proposals (torch.Tensor): Filtered proposals after NMS, shape [post_nms_top_n, 4].
    """
    proposals = []
    scores = []

    for level, (obj, box_reg, anchor) in enumerate(zip(objectness, box_regression, anchors)):
        # Flatten objectness and box_regression predictions
        obj = obj.view(-1)  # Shape: [num_anchors * H * W]
        box_reg = box_reg.view(-1, 4)  # Shape: [num_anchors * H * W, 4]
        anchor = anchor.view(-1, 4)  # Shape: [num_anchors * H * W, 4]

        # Debug prints
        #print(f"Level {level} - Objectness shape: {obj.shape}, Box regression shape: {box_reg.shape}, Anchor shape: {anchor.shape}")

        # Ensure shapes match before decoding
        if obj.shape[0] != box_reg.shape[0] or box_reg.shape[0] != anchor.shape[0]:
            print(f"Shape mismatch at level {level}: obj {obj.shape[0]}, box_reg {box_reg.shape[0]}, anchor {anchor.shape[0]}")
            continue  # Skip this level if there's a mismatch

        # Decode the box regression outputs to generate proposals
        proposals_level = decode_boxes(anchor, box_reg)

        # Clip proposals to image bounds
        proposals_level[:, [0, 2]] = proposals_level[:, [0, 2]].clamp(0, image_size[1])
        proposals_level[:, [1, 3]] = proposals_level[:, [1, 3]].clamp(0, image_size[0])

        # Adjust topk selection to the actual number of available scores
        num_scores = obj.size(0)
        topk = min(pre_nms_top_n, num_scores)  # Use the minimum of available scores and pre_nms_top_n
        topk_indices = obj.topk(topk).indices

        proposals.append(proposals_level[topk_indices])
        scores.append(obj[topk_indices])

    # Concatenate all proposals and scores across FPN levels
    proposals = torch.cat(proposals, dim=0) if proposals else torch.empty(0, 4).to(anchors[0].device)  # Shape: [total_proposals, 4]
    scores = torch.cat(scores, dim=0) if scores else torch.empty(0).to(anchors[0].device)              # Shape: [total_proposals]

    # Debug: check concatenated shapes
    #print("Concatenated proposals shape:", proposals.shape)
    #print("Concatenated scores shape:", scores.shape)

    # Apply NMS to the proposals based on the scores
    if proposals.shape[0] > 0:
        keep_indices = nms(proposals, scores, nms_thresh)
        keep_indices = keep_indices[:post_nms_top_n]
        proposals = proposals[keep_indices]
    else:
        proposals = torch.empty(post_nms_top_n, 4).to(anchors[0].device)

    return proposals






def decode_boxes(anchors, box_deltas):
    """
    Decodes box regression deltas to predicted box coordinates.

    Args:
        anchors (torch.Tensor): Anchor boxes, shape [N, 4].
        box_deltas (torch.Tensor): Box regression deltas, shape [N, 4].

    Returns:
        proposals (torch.Tensor): Decoded proposals, shape [N, 4].
    """
    # Check and reshape box_deltas if the shapes don't align
    # if anchors.shape[0] != box_deltas.shape[0]:
    #     # Reshape box_deltas to match anchors in the first dimension
    #     box_deltas = box_deltas.view(-1, 4)  # Ensures it has [N, 4] shape

    #print("Anchors shape:", anchors.shape)
    #print("Box deltas shape:", box_deltas.shape)

    # Calculate widths, heights, and centers of the anchors
    widths = anchors[:, 2] - anchors[:, 0]
    heights = anchors[:, 3] - anchors[:, 1]
    ctr_x = anchors[:, 0] + 0.5 * widths
    ctr_y = anchors[:, 1] + 0.5 * heights

    # Apply box deltas to anchors
    dx = box_deltas[:, 0]
    dy = box_deltas[:, 1]
    dw = box_deltas[:, 2]
    dh = box_deltas[:, 3]

    pred_ctr_x = ctr_x + dx * widths
    pred_ctr_y = ctr_y + dy * heights
    dw = torch.clamp(dw, min=-10, max=10)  # Adjust the range as necessary
    dh = torch.clamp(dh, min=-10, max=10)

    pred_w = widths * torch.exp(dw)
    pred_h = heights * torch.exp(dh)

    # Calculate the coordinates of the proposed boxes
    proposals = torch.zeros_like(box_deltas)
    proposals[:, 0] = pred_ctr_x - 0.5 * pred_w
    proposals[:, 1] = pred_ctr_y - 0.5 * pred_h
    proposals[:, 2] = pred_ctr_x + 0.5 * pred_w
    proposals[:, 3] = pred_ctr_y + 0.5 * pred_h



    return proposals



In [ ]:
def test_filter_proposals():
    # Dummy inputs based on previous RPN output and anchor generation
    # Assume 9 anchors per spatial location, so 9 * H * W anchors per level
    objectness = [torch.rand(1, 9, 64, 64).view(-1) for _ in range(4)]  # Flattened scores per level
    box_regression = [torch.rand(1, 36, 64, 64).view(-1, 4) for _ in range(4)]  # Shape [num_anchors * H * W, 4]
    anchors = [torch.rand(64 * 64 * 9, 4) for _ in range(4)]  # Shape [num_anchors * H * W, 4] per level

    # Example image size
    image_size = (256, 256)

    # Run filter_proposals
    proposals = filter_proposals(objectness, box_regression, anchors, image_size)

    print("Filtered proposals shape:", proposals.shape)

# Run the test
test_filter_proposals()


Filtered proposals shape: torch.Size([300, 4])


Combined testing of all code so far

In [6]:
def combined_test():
    # Step 1: Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Step 2: Create a dummy input image tensor and move it to the correct device
    dummy_image = torch.rand(1, 3, 256, 256).to(device)

    # Step 3: Pass the image through the backbone to obtain feature maps
    feature_maps = backbone_network(dummy_image)

    # Step 4: Initialize the RPN and generate objectness scores and box regressions
    num_anchors = 9  # Assuming 3 scales and 3 aspect ratios per level
    rpn = MultiLevelRPN(num_anchors=num_anchors).to(device)
    objectness, box_regression = rpn(feature_maps)

    # Step 5: Generate anchors for each feature map level and move anchors to device
    sizes = [(32, 64, 128), (64, 128, 256), (128, 256, 512), (256, 512, 1024)]
    aspect_ratios = [(0.5, 1.0, 2.0)] * 4  # Same aspect ratios for each FPN level
    anchor_gen = AnchorGenerator(sizes, aspect_ratios)
    feature_map_shapes = [(fmap.shape[2], fmap.shape[3]) for fmap in feature_maps.values()]

    # Generate anchors and move them to the correct device
    anchors = [anchor.to(device) for anchor in anchor_gen.generate_anchors(feature_map_shapes)]

    # Step 6: Use the filter_proposals function to generate final proposals
    image_size = (256, 256)
    proposals = filter_proposals(objectness, box_regression, anchors, image_size)

    # Print out the result to verify everything is working together
    print("Combined test - final filtered proposals shape:", proposals.shape)

# Run the combined test
combined_test()


Combined test - final filtered proposals shape: torch.Size([300, 4])


Discriminative RoI Pooling
Function: Extracts discriminative features for classification.

Input: Candidate proposal features from RPN

Output: Pooled RoI features

Description: Uses adaptive weighted pooling to enhance features for classification.

In [7]:


class CustomRoIPooling:
    def __init__(self, output_size=(7, 7), sampling_ratio=2):
        """
        Custom RoI Pooling to handle feature maps with different channel depths.

        Args:
            output_size (tuple): The size (height, width) of the output features after pooling.
            sampling_ratio (int): Sampling ratio for RoIAlign (controls granularity of interpolation).
        """
        self.output_size = output_size
        self.sampling_ratio = sampling_ratio

    def forward(self, feature_maps, proposals, image_size):
        """
        Applies RoI pooling to each feature map level independently.

        Args:
            feature_maps (dict): Dictionary of feature maps from backbone/FPN.
            proposals (torch.Tensor): Proposed regions of interest, shape [num_proposals, 4].
            image_size (tuple): Original image size (height, width).

        Returns:
            pooled_features (torch.Tensor): Combined pooled features.
        """
        pooled_features_list = []

        for level, fmap in feature_maps.items():
            # Perform RoI Align on the current feature map
            pooled = roi_align(
                fmap, [proposals],
                output_size=self.output_size,
                spatial_scale=fmap.shape[-2] / image_size[0],  # Scale based on feature map level
                sampling_ratio=self.sampling_ratio
            )
            pooled_features_list.append(pooled)

        # Concatenate along the channel dimension
        pooled_features = torch.cat(pooled_features_list, dim=1)

        return pooled_features

In [ ]:
# Modify the test function to use CustomRoIPooling
def test_custom_roi_pooling():
    # Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Create a dummy input image tensor and move it to the correct device
    dummy_image = torch.rand(1, 3, 256, 256).to(device)
    print("Dummy image shape:", dummy_image.shape)

    # Pass the image through the backbone to obtain feature maps
    feature_maps = backbone_network(dummy_image)
    for level, fmap in feature_maps.items():
        print(f"Feature map {level} shape:", fmap.shape)

    # Initialize the RPN and generate objectness scores and box regressions
    num_anchors = 9  # Assuming 3 scales and 3 aspect ratios per level
    rpn = MultiLevelRPN(num_anchors=num_anchors).to(device)
    objectness, box_regression = rpn(feature_maps)
    print("Objectness shapes:", [obj.shape for obj in objectness])
    print("Box regression shapes:", [box.shape for box in box_regression])

    # Generate anchors and filter proposals
    sizes = [(32, 64, 128), (64, 128, 256), (128, 256, 512), (256, 512, 1024)]
    aspect_ratios = [(0.5, 1.0, 2.0)] * 4
    anchor_gen = AnchorGenerator(sizes, aspect_ratios)
    feature_map_shapes = [(fmap.shape[2], fmap.shape[3]) for fmap in feature_maps.values()]
    anchors = [anchor.to(device) for anchor in anchor_gen.generate_anchors(feature_map_shapes)]

    image_size = (256, 256)
    proposals = filter_proposals(objectness, box_regression, anchors, image_size)
    print("Filtered proposals shape:", proposals.shape)

    # Initialize and apply custom RoI Pooling
    custom_roi_pooling = CustomRoIPooling(output_size=(7, 7), sampling_ratio=2)
    pooled_features = custom_roi_pooling.forward(feature_maps, proposals, image_size)

    # Print out the shape of the pooled features to verify correctness
    print("Pooled features shape:", pooled_features.shape)

# Run the test
test_custom_roi_pooling()


Dummy image shape: torch.Size([1, 3, 256, 256])
Feature map P3 shape: torch.Size([1, 256, 64, 64])
Feature map P4 shape: torch.Size([1, 512, 32, 32])
Feature map P5 shape: torch.Size([1, 1024, 16, 16])
Feature map P6 shape: torch.Size([1, 2048, 8, 8])
Objectness shapes: [torch.Size([1, 9, 64, 64]), torch.Size([1, 9, 32, 32]), torch.Size([1, 9, 16, 16]), torch.Size([1, 9, 8, 8])]
Box regression shapes: [torch.Size([1, 36, 64, 64]), torch.Size([1, 36, 32, 32]), torch.Size([1, 36, 16, 16]), torch.Size([1, 36, 8, 8])]
Filtered proposals shape: torch.Size([300, 4])
Pooled features shape: torch.Size([300, 3840, 7, 7])


Dense Local Regeression

In [8]:


class DenseLocalRegression(nn.Module):
    def __init__(self, input_channels, output_size=4, intermediate_dim=512):
        """
        Enhanced Dense Local Regression with convolutional layers and attention.

        Args:
            input_channels (int): Number of input channels from pooled features.
            output_size (int): Number of outputs per proposal (4 for bbox regression: dx, dy, dw, dh).
            intermediate_dim (int): Dimension for intermediate fully connected layers.
        """
        super(DenseLocalRegression, self).__init__()

        # Convolutional layers to process spatial features
        self.conv1 = nn.Conv2d(input_channels, 256, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(256)
        self.conv2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(256)

        # Spatial Attention Layer
        self.spatial_attn_conv = nn.Conv2d(256, 1, kernel_size=1)  # 1x1 Conv for spatial attention

        # Fully connected regression head
        self.fc1 = nn.Linear(256 * 7 * 7, intermediate_dim)
        self.fc2 = nn.Linear(intermediate_dim, intermediate_dim)
        self.fc_out = nn.Linear(intermediate_dim, output_size)

        # Dropout for regularization
        self.dropout = nn.Dropout(0.3)

        self.relu = nn.ReLU()

    def forward(self, pooled_features):
        """
        Forward pass through the Enhanced Dense Local Regression module.

        Args:
            pooled_features (torch.Tensor): RoI pooled features, shape [num_proposals, C, H, W].

        Returns:
            torch.Tensor: Adjustments for bounding boxes, shape [num_proposals, 4].
        """
        # Step 1: Apply initial convolutions and batch normalization
        x = self.relu(self.bn1(self.conv1(pooled_features)))
        x = self.relu(self.bn2(self.conv2(x)))

        # Step 2: Apply spatial attention
        spatial_attn = torch.sigmoid(self.spatial_attn_conv(x))  # Shape: [num_proposals, 1, H, W]
        x = x * spatial_attn  # Element-wise multiplication with attention map

        # Step 3: Flatten the features for fully connected layers
        x = x.view(x.size(0), -1)  # Shape: [num_proposals, 256 * 7 * 7]

        # Step 4: Apply fully connected layers with dropout for regression
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        bbox_deltas = self.fc_out(x)  # Shape: [num_proposals, 4]

        return bbox_deltas


In [ ]:
def test_dense_local_regression():
    # Step 1: Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Step 2: Create a dummy input image tensor and move it to the correct device
    dummy_image = torch.rand(1, 3, 256, 256).to(device)
    print("Dummy image shape:", dummy_image.shape)

    # Step 3: Pass the image through the backbone to obtain feature maps
    feature_maps = backbone_network(dummy_image)
    for level, fmap in feature_maps.items():
        print(f"Feature map {level} shape:", fmap.shape)

    # Step 4: Initialize the RPN and generate objectness scores and box regressions
    num_anchors = 9  # Assuming 3 scales and 3 aspect ratios per level
    rpn = MultiLevelRPN(num_anchors=num_anchors).to(device)
    objectness, box_regression = rpn(feature_maps)
    print("Objectness shapes:", [obj.shape for obj in objectness])
    print("Box regression shapes:", [box.shape for box in box_regression])

    # Step 5: Generate anchors and filter proposals
    sizes = [(32, 64, 128), (64, 128, 256), (128, 256, 512), (256, 512, 1024)]
    aspect_ratios = [(0.5, 1.0, 2.0)] * 4
    anchor_gen = AnchorGenerator(sizes, aspect_ratios)
    feature_map_shapes = [(fmap.shape[2], fmap.shape[3]) for fmap in feature_maps.values()]
    anchors = [anchor.to(device) for anchor in anchor_gen.generate_anchors(feature_map_shapes)]

    image_size = (256, 256)
    proposals = filter_proposals(objectness, box_regression, anchors, image_size)
    print("Filtered proposals shape:", proposals.shape)

    # Step 6: Initialize and apply custom RoI Pooling
    custom_roi_pooling = CustomRoIPooling(output_size=(7, 7), sampling_ratio=2)
    pooled_features = custom_roi_pooling.forward(feature_maps, proposals, image_size)
    print("Pooled features shape:", pooled_features.shape)

    # Step 7: Initialize Dense Local Regression module and apply it to the pooled features
    input_channels = pooled_features.shape[1]  # This should be the combined channels from FPN levels
    dense_local_regression = DenseLocalRegression(input_channels=input_channels).to(device)
    bbox_deltas = dense_local_regression(pooled_features)
    print("Bounding box deltas shape:", bbox_deltas.shape)

    # Check if output has correct shape [num_proposals, 4]
    assert bbox_deltas.shape == (pooled_features.shape[0], 4), \
        f"Expected shape {(pooled_features.shape[0], 4)}, but got {bbox_deltas.shape}"

    print("Test passed: Dense Local Regression module outputs correctly shaped bbox deltas.")

# Run the test
test_dense_local_regression()


Dummy image shape: torch.Size([1, 3, 256, 256])
Feature map P3 shape: torch.Size([1, 256, 64, 64])
Feature map P4 shape: torch.Size([1, 512, 32, 32])
Feature map P5 shape: torch.Size([1, 1024, 16, 16])
Feature map P6 shape: torch.Size([1, 2048, 8, 8])
Objectness shapes: [torch.Size([1, 9, 64, 64]), torch.Size([1, 9, 32, 32]), torch.Size([1, 9, 16, 16]), torch.Size([1, 9, 8, 8])]
Box regression shapes: [torch.Size([1, 36, 64, 64]), torch.Size([1, 36, 32, 32]), torch.Size([1, 36, 16, 16]), torch.Size([1, 36, 8, 8])]
Filtered proposals shape: torch.Size([300, 4])
Pooled features shape: torch.Size([300, 3840, 7, 7])
Bounding box deltas shape: torch.Size([300, 4])
Test passed: Dense Local Regression module outputs correctly shaped bbox deltas.


Bounding Box Refinement

In [38]:
import torch

class BoundingBoxRefinement:
    def __init__(self):
        """
        Initializes the Bounding Box Refinement module.
        """
        pass  # No specific layers required as it's a geometric transformation

    def refine_boxes(self, proposals, bbox_deltas, image_size):
        """
        Refines bounding boxes by applying bbox deltas to proposals.

        Args:
            proposals (torch.Tensor): Tensor of proposals, shape [num_proposals, 4].
            bbox_deltas (torch.Tensor): Tensor of deltas to be applied to proposals, shape [num_proposals, 4].
            image_size (tuple): The size of the image (height, width).

        Returns:
            torch.Tensor: Refined bounding boxes with values clamped to image boundaries.
        """
        refined_boxes = proposals + bbox_deltas

        # Clamping refined boxes to stay within image boundaries
        refined_boxes = torch.stack([
            torch.clamp(refined_boxes[:, 0], 0, image_size[1]),  # x1
            torch.clamp(refined_boxes[:, 1], 0, image_size[0]),  # y1
            torch.clamp(refined_boxes[:, 2], 0, image_size[1]),  # x2
            torch.clamp(refined_boxes[:, 3], 0, image_size[0])   # y2
        ], dim=1)

        return refined_boxes



In [39]:
def test_bounding_box_refinement():
    # Step 1: Create dummy proposals and box deltas
    num_proposals = 300
    image_size = (256, 256)

    proposals = torch.rand(num_proposals, 4) * 256  # Random proposals within image bounds
    proposals[:, 2:] += proposals[:, :2]  # Ensure x2 > x1 and y2 > y1

    box_deltas = torch.randn(num_proposals, 4) * 0.1  # Small random deltas for refinement

    # Step 2: Initialize and apply BoundingBoxRefinement
    bbox_refiner = BoundingBoxRefinement()
    refined_boxes = bbox_refiner.refine_boxes(proposals, box_deltas, image_size)

    # Step 3: Print shapes and check if boxes are within image boundaries
    print("Refined boxes shape:", refined_boxes.shape)  # Expecting [num_proposals, 4]
    assert refined_boxes.shape == (num_proposals, 4), "Shape mismatch in refined boxes."

    # Check that all boxes are within image boundaries
    assert torch.all(refined_boxes[:, 0] >= 0) and torch.all(refined_boxes[:, 1] >= 0), \
        "Refined boxes have coordinates less than 0."
    assert torch.all(refined_boxes[:, 2] <= image_size[1]) and torch.all(refined_boxes[:, 3] <= image_size[0]), \
        "Refined boxes exceed image boundaries."

    print("Test passed: BoundingBoxRefinement outputs correctly shaped and bounded boxes.")

# Run the test
test_bounding_box_refinement()


Refined boxes shape: torch.Size([300, 4])
Test passed: BoundingBoxRefinement outputs correctly shaped and bounded boxes.


Discriminative RoI Pooling

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DiscriminativeRoIPooling(nn.Module):
    def __init__(self, input_channels, output_channels=256):
        """
        Enhanced Discriminative RoI Pooling with an attention mechanism and additional layers.

        Args:
            input_channels (int): Number of input channels from pooled features.
            output_channels (int): Number of output channels for enhanced features.
        """
        super(DiscriminativeRoIPooling, self).__init__()

        # Initial Convolutional Layers to refine features
        self.conv1 = nn.Conv2d(input_channels, output_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(output_channels)
        self.conv2 = nn.Conv2d(output_channels, output_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(output_channels)

        # Spatial Attention Layer
        self.spatial_attn_conv = nn.Conv2d(output_channels, 1, kernel_size=1)  # 1x1 Conv for spatial attention

        # Channel Attention Layer (Squeeze-and-Excitation)
        self.channel_attn_fc1 = nn.Linear(output_channels, output_channels // 4)  # Bottleneck layer
        self.channel_attn_fc2 = nn.Linear(output_channels // 4, output_channels)

        self.relu = nn.ReLU()

    def forward(self, pooled_features):
        """
        Forward pass through the enhanced Discriminative RoI Pooling module.

        Args:
            pooled_features (torch.Tensor): Pooled features from RoI pooling, shape [num_proposals, C, H, W].

        Returns:
            torch.Tensor: Enhanced features with attention, shape [num_proposals, output_channels, H, W].
        """
        # Step 1: Initial convolutions and batch normalization
        x = self.relu(self.bn1(self.conv1(pooled_features)))
        x = self.relu(self.bn2(self.conv2(x)))

        # Step 2: Spatial Attention
        spatial_attn = torch.sigmoid(self.spatial_attn_conv(x))  # Shape: [num_proposals, 1, H, W]
        x = x * spatial_attn  # Apply spatial attention

        # Step 3: Channel Attention (Squeeze-and-Excitation)
        b, c, h, w = x.size()
        channel_attn = F.adaptive_avg_pool2d(x, 1).view(b, c)  # Global average pooling to [num_proposals, C]
        channel_attn = self.relu(self.channel_attn_fc1(channel_attn))
        channel_attn = torch.sigmoid(self.channel_attn_fc2(channel_attn))  # Shape: [num_proposals, C]
        channel_attn = channel_attn.view(b, c, 1, 1)  # Reshape to broadcast
        x = x * channel_attn  # Apply channel attention

        return x


In [ ]:
def test_discriminative_roi_pooling():
    # Step 1: Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Step 2: Create a dummy input image tensor and move it to the correct device
    dummy_image = torch.rand(1, 3, 256, 256).to(device)
    print("Dummy image shape:", dummy_image.shape)

    # Step 3: Pass the image through the backbone to obtain feature maps
    feature_maps = backbone_network(dummy_image)
    for level, fmap in feature_maps.items():
        print(f"Feature map {level} shape:", fmap.shape)

    # Step 4: Initialize the RPN and generate objectness scores and box regressions
    num_anchors = 9  # Assuming 3 scales and 3 aspect ratios per level
    rpn = MultiLevelRPN(num_anchors=num_anchors).to(device)
    objectness, box_regression = rpn(feature_maps)
    print("Objectness shapes:", [obj.shape for obj in objectness])
    print("Box regression shapes:", [box.shape for box in box_regression])

    # Step 5: Generate anchors and filter proposals
    sizes = [(32, 64, 128), (64, 128, 256), (128, 256, 512), (256, 512, 1024)]
    aspect_ratios = [(0.5, 1.0, 2.0)] * 4
    anchor_gen = AnchorGenerator(sizes, aspect_ratios)
    feature_map_shapes = [(fmap.shape[2], fmap.shape[3]) for fmap in feature_maps.values()]
    anchors = [anchor.to(device) for anchor in anchor_gen.generate_anchors(feature_map_shapes)]

    image_size = (256, 256)
    proposals = filter_proposals(objectness, box_regression, anchors, image_size)
    print("Filtered proposals shape:", proposals.shape)

    # Step 6: Initialize and apply custom RoI Pooling
    custom_roi_pooling = CustomRoIPooling(output_size=(7, 7), sampling_ratio=2)
    pooled_features = custom_roi_pooling.forward(feature_maps, proposals, image_size)
    print("Pooled features shape:", pooled_features.shape)

    # Step 7: Initialize Discriminative RoI Pooling module and apply it
    input_channels = pooled_features.shape[1]
    discriminative_roi_pooling = DiscriminativeRoIPooling(input_channels=input_channels, output_channels=256).to(device)
    enhanced_features = discriminative_roi_pooling(pooled_features)
    print("Enhanced features shape:", enhanced_features.shape)

    # Check if output has correct shape
    assert enhanced_features.shape == (pooled_features.shape[0], 256, 7, 7), \
        f"Expected shape {(pooled_features.shape[0], 256, 7, 7)}, but got {enhanced_features.shape}"

    print("Test passed: Discriminative RoI Pooling module outputs correctly shaped enhanced features.")

# Run the test
test_discriminative_roi_pooling()


Dummy image shape: torch.Size([1, 3, 256, 256])
Feature map P3 shape: torch.Size([1, 256, 64, 64])
Feature map P4 shape: torch.Size([1, 512, 32, 32])
Feature map P5 shape: torch.Size([1, 1024, 16, 16])
Feature map P6 shape: torch.Size([1, 2048, 8, 8])
Objectness shapes: [torch.Size([1, 9, 64, 64]), torch.Size([1, 9, 32, 32]), torch.Size([1, 9, 16, 16]), torch.Size([1, 9, 8, 8])]
Box regression shapes: [torch.Size([1, 36, 64, 64]), torch.Size([1, 36, 32, 32]), torch.Size([1, 36, 16, 16]), torch.Size([1, 36, 8, 8])]
Filtered proposals shape: torch.Size([300, 4])
Pooled features shape: torch.Size([300, 3840, 7, 7])
Enhanced features shape: torch.Size([300, 256, 7, 7])
Test passed: Discriminative RoI Pooling module outputs correctly shaped enhanced features.


Binary Overlap Prediction

In [11]:
import torch
import torch.nn as nn

class BinaryOverlapPrediction(nn.Module):
    def __init__(self, input_channels):
        """
        Initializes the Binary Overlap Prediction module.

        Args:
            input_channels (int): Number of channels in the input features.
        """
        super(BinaryOverlapPrediction, self).__init__()

        # Fully connected layers for overlap prediction
        self.fc1 = nn.Linear(input_channels * 7 * 7, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc_overlap = nn.Linear(512, 1)  # Single output per region proposal

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()  # Output probability for overlap

    def forward(self, pooled_features):
        """
        Forward pass through the Binary Overlap Prediction module.

        Args:
            pooled_features (torch.Tensor): Pooled RoI features, shape [num_proposals, C, H, W].

        Returns:
            torch.Tensor: Overlap scores per proposal, shape [num_proposals, 1].
        """
        # Flatten features and pass through the fully connected layers
        x = pooled_features.view(pooled_features.size(0), -1)  # Flatten to [num_proposals, C*H*W]
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        overlap_scores = self.sigmoid(self.fc_overlap(x))  # Probability of overlap [num_proposals, 1]

        return overlap_scores


In [ ]:
def test_binary_overlap_prediction():
    # Step 1: Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Step 2: Create a dummy input image tensor and move it to the correct device
    dummy_image = torch.rand(1, 3, 256, 256).to(device)
    print("Dummy image shape:", dummy_image.shape)

    # Step 3: Pass the image through the backbone to obtain feature maps
    feature_maps = backbone_network(dummy_image)
    for level, fmap in feature_maps.items():
        print(f"Feature map {level} shape:", fmap.shape)

    # Step 4: Initialize the RPN and generate objectness scores and box regressions
    num_anchors = 9  # Assuming 3 scales and 3 aspect ratios per level
    rpn = MultiLevelRPN(num_anchors=num_anchors).to(device)
    objectness, box_regression = rpn(feature_maps)
    print("Objectness shapes:", [obj.shape for obj in objectness])
    print("Box regression shapes:", [box.shape for box in box_regression])

    # Step 5: Generate anchors and filter proposals
    sizes = [(32, 64, 128), (64, 128, 256), (128, 256, 512), (256, 512, 1024)]
    aspect_ratios = [(0.5, 1.0, 2.0)] * 4
    anchor_gen = AnchorGenerator(sizes, aspect_ratios)
    feature_map_shapes = [(fmap.shape[2], fmap.shape[3]) for fmap in feature_maps.values()]
    anchors = [anchor.to(device) for anchor in anchor_gen.generate_anchors(feature_map_shapes)]

    image_size = (256, 256)
    proposals = filter_proposals(objectness, box_regression, anchors, image_size)
    print("Filtered proposals shape:", proposals.shape)

    # Step 6: Initialize and apply custom RoI Pooling
    custom_roi_pooling = CustomRoIPooling(output_size=(7, 7), sampling_ratio=2)
    pooled_features = custom_roi_pooling.forward(feature_maps, proposals, image_size)
    print("Pooled features shape:", pooled_features.shape)

    # Step 7: Initialize Binary Overlap Prediction module and apply it
    input_channels = pooled_features.shape[1]
    binary_overlap_prediction = BinaryOverlapPrediction(input_channels=input_channels).to(device)
    overlap_scores = binary_overlap_prediction(pooled_features)
    print("Overlap scores shape:", overlap_scores.shape)

    # Check if output has correct shape [num_proposals, 1]
    assert overlap_scores.shape == (pooled_features.shape[0], 1), \
        f"Expected shape {(pooled_features.shape[0], 1)}, but got {overlap_scores.shape}"

    # Check if values are within [0, 1] range
    assert (overlap_scores >= 0).all() and (overlap_scores <= 1).all(), "Overlap scores are out of range [0, 1]"

    print("Test passed: Binary Overlap Prediction module outputs correctly shaped and bounded overlap scores.")

# Run the test
test_binary_overlap_prediction()


Dummy image shape: torch.Size([1, 3, 256, 256])
Feature map P3 shape: torch.Size([1, 256, 64, 64])
Feature map P4 shape: torch.Size([1, 512, 32, 32])
Feature map P5 shape: torch.Size([1, 1024, 16, 16])
Feature map P6 shape: torch.Size([1, 2048, 8, 8])
Objectness shapes: [torch.Size([1, 9, 64, 64]), torch.Size([1, 9, 32, 32]), torch.Size([1, 9, 16, 16]), torch.Size([1, 9, 8, 8])]
Box regression shapes: [torch.Size([1, 36, 64, 64]), torch.Size([1, 36, 32, 32]), torch.Size([1, 36, 16, 16]), torch.Size([1, 36, 8, 8])]
Filtered proposals shape: torch.Size([300, 4])
Pooled features shape: torch.Size([300, 3840, 7, 7])
Overlap scores shape: torch.Size([300, 1])
Test passed: Binary Overlap Prediction module outputs correctly shaped and bounded overlap scores.


Object Classification

In [12]:
import torch.nn as nn

class ObjectClassification(nn.Module):
    def __init__(self, input_channels, num_classes):
        """
        Initializes the Object Classification module with enhanced architecture.

        Args:
            input_channels (int): Number of input channels (should match the pooled feature map's channel dimension).
            num_classes (int): Total number of classes, including one for background.
        """
        super(ObjectClassification, self).__init__()

        # Fully connected layers for classification with BatchNorm and Dropout
        self.fc1 = nn.Linear(input_channels * 7 * 7, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.dropout1 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.dropout2 = nn.Dropout(0.5)

        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.dropout3 = nn.Dropout(0.3)

        self.fc_out = nn.Linear(256, num_classes)  # Output layer with num_classes outputs

        # Activation functions
        self.relu = nn.LeakyReLU(0.1)  # Leaky ReLU for better gradient flow
        self.softmax = nn.Softmax(dim=1)  # Apply softmax along the class dimension

    def forward(self, pooled_features):
        """
        Forward pass through the enhanced Object Classification module.

        Args:
            pooled_features (torch.Tensor): RoI pooled features, shape [num_proposals, C, H, W].

        Returns:
            torch.Tensor: Class probabilities, shape [num_proposals, num_classes].
        """
        # Flatten the features for fully connected layers
        x = pooled_features.view(pooled_features.size(0), -1)  # Shape: [num_proposals, C*H*W]

        # Layer 1: Fully connected + BatchNorm + Dropout + Activation
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)

        # Layer 2: Fully connected + BatchNorm + Dropout + Activation
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)

        # Layer 3: Additional Fully connected + BatchNorm + Dropout + Activation
        x = self.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)

        # Output layer with softmax activation for class probabilities
        class_logits = self.fc_out(x)  # Shape: [num_proposals, num_classes]
        class_probs = self.softmax(class_logits)  # Softmax to get class probabilities

        return class_probs


In [ ]:
def test_object_classification_with_binary_overlap():
    # Step 1: Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Step 2: Create a dummy input image tensor and move it to the correct device
    dummy_image = torch.rand(1, 3, 256, 256).to(device)
    print("Dummy image shape:", dummy_image.shape)

    # Step 3: Pass the image through the backbone to obtain feature maps
    feature_maps = backbone_network(dummy_image)
    for level, fmap in feature_maps.items():
        print(f"Feature map {level} shape:", fmap.shape)

    # Step 4: Initialize the RPN and generate objectness scores and box regressions
    num_anchors = 9  # Assuming 3 scales and 3 aspect ratios per level
    rpn = MultiLevelRPN(num_anchors=num_anchors).to(device)
    objectness, box_regression = rpn(feature_maps)
    print("Objectness shapes:", [obj.shape for obj in objectness])
    print("Box regression shapes:", [box.shape for box in box_regression])

    # Step 5: Generate anchors and filter proposals
    sizes = [(32, 64, 128), (64, 128, 256), (128, 256, 512), (256, 512, 1024)]
    aspect_ratios = [(0.5, 1.0, 2.0)] * 4
    anchor_gen = AnchorGenerator(sizes, aspect_ratios)
    feature_map_shapes = [(fmap.shape[2], fmap.shape[3]) for fmap in feature_maps.values()]
    anchors = [anchor.to(device) for anchor in anchor_gen.generate_anchors(feature_map_shapes)]

    image_size = (256, 256)
    proposals = filter_proposals(objectness, box_regression, anchors, image_size)
    print("Filtered proposals shape:", proposals.shape)

    # Step 6: Initialize and apply custom RoI Pooling
    custom_roi_pooling = CustomRoIPooling(output_size=(7, 7), sampling_ratio=2)
    pooled_features = custom_roi_pooling.forward(feature_maps, proposals, image_size)
    print("Pooled features shape:", pooled_features.shape)

    # Step 7: Initialize Discriminative RoI Pooling module and apply it
    input_channels = pooled_features.shape[1]
    discriminative_roi_pooling = DiscriminativeRoIPooling(input_channels=input_channels, output_channels=256).to(device)
    enhanced_features = discriminative_roi_pooling(pooled_features)
    print("Enhanced features shape:", enhanced_features.shape)

    # Step 8: Initialize and apply Binary Overlap Prediction
    binary_overlap = BinaryOverlapPrediction(input_channels=256).to(device)
    overlap_scores = binary_overlap(enhanced_features)
    print("Overlap scores shape:", overlap_scores.shape)  # Expecting [num_proposals, 1]

    # Check if overlap scores have correct shape
    assert overlap_scores.shape == (enhanced_features.shape[0], 1), \
        f"Expected shape {(enhanced_features.shape[0], 1)}, but got {overlap_scores.shape}"

    # Step 9: Initialize and test the Object Classification module
    num_classes = 21  # Assuming 20 object classes + 1 background class
    object_classifier = ObjectClassification(input_channels=256, num_classes=num_classes).to(device)
    class_probs = object_classifier(enhanced_features)

    # Print shapes to verify correctness
    print("Class probabilities shape:", class_probs.shape)  # Expecting [num_proposals, num_classes]

    # Check if output has correct shape
    assert class_probs.shape == (enhanced_features.shape[0], num_classes), \
        f"Test failed: Expected shape {(enhanced_features.shape[0], num_classes)}, but got {class_probs.shape}"

    print("Test passed: Object Classification module outputs correctly shaped class probabilities.")
    print("Test passed: Binary Overlap Prediction module outputs correctly shaped overlap scores.")

# Run the test
test_object_classification_with_binary_overlap()


Dummy image shape: torch.Size([1, 3, 256, 256])
Feature map P3 shape: torch.Size([1, 256, 64, 64])
Feature map P4 shape: torch.Size([1, 512, 32, 32])
Feature map P5 shape: torch.Size([1, 1024, 16, 16])
Feature map P6 shape: torch.Size([1, 2048, 8, 8])
Objectness shapes: [torch.Size([1, 9, 64, 64]), torch.Size([1, 9, 32, 32]), torch.Size([1, 9, 16, 16]), torch.Size([1, 9, 8, 8])]
Box regression shapes: [torch.Size([1, 36, 64, 64]), torch.Size([1, 36, 32, 32]), torch.Size([1, 36, 16, 16]), torch.Size([1, 36, 8, 8])]
Filtered proposals shape: torch.Size([300, 4])
Pooled features shape: torch.Size([300, 3840, 7, 7])
Enhanced features shape: torch.Size([300, 256, 7, 7])
Overlap scores shape: torch.Size([300, 1])
Class probabilities shape: torch.Size([300, 21])
Test passed: Object Classification module outputs correctly shaped class probabilities.
Test passed: Binary Overlap Prediction module outputs correctly shaped overlap scores.


Instance Segmentation - Mask Prediction

In [101]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MaskPrediction(nn.Module):
    def __init__(self, input_channels, mask_size=28, num_classes=1):
        """
        Initializes the Mask Prediction module for instance segmentation.

        Args:
            input_channels (int): Number of input channels (should match the pooled feature map's channel dimension).
            mask_size (int): The height and width of the output mask (e.g., 28x28).
            num_classes (int): Number of classes. Set to 1 for binary masks or higher for class-specific masks.
        """
        super(MaskPrediction, self).__init__()

        # Convolutional layers to process the pooled RoI features
        self.conv1 = nn.Conv2d(input_channels, 256, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        # First upsample layer to increase spatial dimensions
        self.deconv = nn.ConvTranspose2d(256, 256, kernel_size=2, stride=2)

        # Final layer to output the mask with the desired size
        self.conv_out = nn.Conv2d(256, num_classes, kernel_size=1)

        # Activation function
        self.relu = nn.ReLU()

        # Target mask size
        self.mask_size = mask_size

    def forward(self, pooled_features):
        """
        Forward pass through the Mask Prediction module.

        Args:
            pooled_features (torch.Tensor): RoI pooled features, shape [num_proposals, C, H, W].

        Returns:
            torch.Tensor: Binary masks for each proposal, shape [num_proposals, num_classes, mask_size, mask_size].
        """
        # Target shape for pooled features (e.g., 14x14)
        target_height, target_width = self.mask_size // 2, self.mask_size // 2

        # Pad or crop pooled_features to the target shape if necessary
        pooled_features = self.pad_or_crop(pooled_features, target_height, target_width)

        # Forward through the convolutional layers
        x = self.relu(self.conv1(pooled_features))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))

        # First upsample step
        x = self.relu(self.deconv(x))

        # Final conv layer for mask prediction
        mask_logits = self.conv_out(x)

        # Upsample to target size (28x28) if necessary
        if mask_logits.shape[2] != self.mask_size or mask_logits.shape[3] != self.mask_size:
            mask_logits = F.interpolate(mask_logits, size=(self.mask_size, self.mask_size), mode='bilinear', align_corners=False)

        # Apply sigmoid for binary mask or softmax for multi-class mask
        if mask_logits.shape[1] == 1:
            mask_probs = torch.sigmoid(mask_logits)
        else:
            mask_probs = F.softmax(mask_logits, dim=1)

        return mask_probs

    def pad_or_crop(self, features, target_height, target_width):
        """
        Pad or crop features to the target height and width.

        Args:
            features (torch.Tensor): Input tensor of shape [N, C, H, W].
            target_height (int): Target height.
            target_width (int): Target width.

        Returns:
            torch.Tensor: Tensor of shape [N, C, target_height, target_width].
        """
        _, _, h, w = features.shape
        pad_h = max(0, target_height - h)
        pad_w = max(0, target_width - w)

        # If padding is needed, pad the height and width dimensions
        if pad_h > 0 or pad_w > 0:
            features = F.pad(features, (0, pad_w, 0, pad_h), mode='constant', value=0)

        # Crop if dimensions are larger than target
        if h > target_height or w > target_width:
            features = features[:, :, :target_height, :target_width]

        return features


In [102]:
def test_mask_prediction():
    # Step 1: Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Step 2: Create a dummy input image tensor and move it to the correct device
    dummy_image = torch.rand(1, 3, 256, 256).to(device)
    print("Dummy image shape:", dummy_image.shape)

    # Step 3: Pass the image through the backbone to obtain feature maps
    feature_maps = backbone_network(dummy_image)
    for level, fmap in feature_maps.items():
        print(f"Feature map {level} shape:", fmap.shape)

    # Step 4: Initialize the RPN and generate objectness scores and box regressions
    num_anchors = 9  # Assuming 3 scales and 3 aspect ratios per level
    rpn = MultiLevelRPN(num_anchors=num_anchors).to(device)
    objectness, box_regression = rpn(feature_maps)
    print("Objectness shapes:", [obj.shape for obj in objectness])
    print("Box regression shapes:", [box.shape for box in box_regression])

    # Step 5: Generate anchors and filter proposals
    sizes = [(32, 64, 128), (64, 128, 256), (128, 256, 512), (256, 512, 1024)]
    aspect_ratios = [(0.5, 1.0, 2.0)] * 4
    anchor_gen = AnchorGenerator(sizes, aspect_ratios)
    feature_map_shapes = [(fmap.shape[2], fmap.shape[3]) for fmap in feature_maps.values()]
    anchors = [anchor.to(device) for anchor in anchor_gen.generate_anchors(feature_map_shapes)]

    image_size = (256, 256)
    proposals = filter_proposals(objectness, box_regression, anchors, image_size)
    print("Filtered proposals shape:", proposals.shape)

    # Step 6: Initialize and apply custom RoI Pooling
    custom_roi_pooling = CustomRoIPooling(output_size=(7, 7), sampling_ratio=2)
    pooled_features = custom_roi_pooling.forward(feature_maps, proposals, image_size)
    print("Pooled features shape:", pooled_features.shape)

    # Step 7: Initialize Discriminative RoI Pooling module and apply it
    input_channels = pooled_features.shape[1]
    print("Input channels for Discriminative RoI Pooling:", input_channels)
    discriminative_roi_pooling = DiscriminativeRoIPooling(input_channels=input_channels, output_channels=256).to(device)
    enhanced_features = discriminative_roi_pooling(pooled_features)
    print("Enhanced features shape:", enhanced_features.shape)

    # Step 8: Initialize and apply Mask Prediction module
    mask_size = 28
    mask_prediction = MaskPrediction(input_channels=256, mask_size=mask_size, num_classes=1).to(device)
    masks = mask_prediction(enhanced_features)
    print("Masks shape:", masks.shape)

    # Check if output has correct shape
    assert masks.shape == (enhanced_features.shape[0], 1, mask_size, mask_size), \
        f"Expected shape {(enhanced_features.shape[0], 1, mask_size, mask_size)}, but got {masks.shape}"

    print("Test passed: Mask Prediction module outputs correctly shaped masks.")

# Run the test
test_mask_prediction()


Dummy image shape: torch.Size([1, 3, 256, 256])
Feature map P3 shape: torch.Size([1, 256, 64, 64])
Feature map P4 shape: torch.Size([1, 512, 32, 32])
Feature map P5 shape: torch.Size([1, 1024, 16, 16])
Feature map P6 shape: torch.Size([1, 2048, 8, 8])
Objectness shapes: [torch.Size([1, 9, 64, 64]), torch.Size([1, 9, 32, 32]), torch.Size([1, 9, 16, 16]), torch.Size([1, 9, 8, 8])]
Box regression shapes: [torch.Size([1, 36, 64, 64]), torch.Size([1, 36, 32, 32]), torch.Size([1, 36, 16, 16]), torch.Size([1, 36, 8, 8])]
Filtered proposals shape: torch.Size([300, 4])
Pooled features shape: torch.Size([300, 3840, 7, 7])
Input channels for Discriminative RoI Pooling: 3840
Enhanced features shape: torch.Size([300, 256, 7, 7])
Masks shape: torch.Size([300, 1, 28, 28])
Test passed: Mask Prediction module outputs correctly shaped masks.


Loss Functions

In [ ]:
# def classification_loss(pred_class_probs, target_category_id):
#     """
#     Calculate the cross-entropy loss for the category classification.

#     Parameters:
#     - pred_class_probs (torch.Tensor): Tensor of shape (num_classes,) representing the predicted
#                                        probability distribution over classes for a specific proposal.
#     - target_category_id (int): The ground-truth category ID as an integer.

#     Returns:
#     - loss (torch.Tensor): The computed cross-entropy loss.
#     """
#     # Ensure pred_class_probs is of shape (1, num_classes) for CrossEntropyLoss
#     pred_class_probs = pred_class_probs.unsqueeze(0)  # Shape becomes (1, num_classes)

#     # Convert target_category_id to a tensor of shape [1]
#     target_class = torch.tensor([target_category_id], dtype=torch.long, device=pred_class_probs.device)

#     # Compute cross-entropy loss
#     loss = F.cross_entropy(pred_class_probs, target_class)

#     return loss

# def false_positive_loss(pred_boxes, pred_objectness, ground_truth_boxes, iou_threshold=0.3):
#     # Compute IoUs between predicted boxes and ground truth boxes
#     ious = compute_iou_matrix(pred_boxes, ground_truth_boxes)  # Custom function
#     max_iou, _ = ious.max(dim=1)

#     # False positives: Proposals with max IoU below the threshold
#     false_positive_mask = (max_iou < iou_threshold).float()
#     fp_loss = false_positive_mask * pred_objectness
#     return fp_loss.mean()


# def iou_loss(pred_boxes, target_boxes):
#     # Calculate intersection
#     inter_x1 = torch.max(pred_boxes[:, 0], target_boxes[:, 0])
#     inter_y1 = torch.max(pred_boxes[:, 1], target_boxes[:, 1])
#     inter_x2 = torch.min(pred_boxes[:, 2], target_boxes[:, 2])
#     inter_y2 = torch.min(pred_boxes[:, 3], target_boxes[:, 3])

#     inter_area = torch.clamp(inter_x2 - inter_x1, min=0) * torch.clamp(inter_y2 - inter_y1, min=0)

#     # Calculate areas of each box and union
#     pred_area = (pred_boxes[:, 2] - pred_boxes[:, 0]) * (pred_boxes[:, 3] - pred_boxes[:, 1])
#     target_area = (target_boxes[:, 2] - target_boxes[:, 0]) * (target_boxes[:, 3] - target_boxes[:, 1])
#     union_area = pred_area + target_area - inter_area

#     iou = inter_area / union_area
#     return 1 - iou.mean()  # IoU loss: 1 - mean IoU


# import torch.nn.functional as F




# def objectness_loss(pred_objectness, target_objectness):
#     # Calculate binary cross-entropy loss between predicted and target objectness
#     return F.binary_cross_entropy(pred_objectness, target_objectness)

# import torch
# import torch.nn.functional as F

# def mask_loss(pred_masks, target_masks):
#     """
#     Calculate binary cross-entropy loss between predicted and target masks for multiple objects in an image.

#     Args:
#         pred_masks (torch.Tensor): Predicted masks, shape [num_proposals, mask_size, mask_size].
#         target_masks (list of torch.Tensor): List of ground truth masks, where each mask tensor has shape [mask_size, mask_size].

#     Returns:
#         torch.Tensor: Average mask loss (binary cross-entropy loss) across all objects.
#     """
#     # Ensure all elements in target_masks are tensors and move to the same device as pred_masks
#     target_masks = [mask.to(pred_masks.device) if isinstance(mask, torch.Tensor) else torch.tensor(mask, device=pred_masks.device)
#                     for mask in target_masks]

#     # Make sure target_masks is a stacked tensor with the same number of masks as predictions if possible
#     target_masks = torch.stack(target_masks)  # Shape: [num_objects, mask_size, mask_size]

#     # If the shapes differ (e.g., more predictions than ground truth objects), adjust to match the smaller set
#     num_objects = min(pred_masks.shape[0], target_masks.shape[0])
#     pred_masks = pred_masks[:num_objects]
#     target_masks = target_masks[:num_objects]

#     # Compute binary cross-entropy loss for each mask
#     individual_losses = []
#     for pred_mask, target_mask in zip(pred_masks, target_masks):
#         individual_loss = F.binary_cross_entropy(pred_mask, target_mask)
#         individual_losses.append(individual_loss)

#     # Average the loss over all masks
#     return torch.mean(torch.stack(individual_losses)) if individual_losses else torch.tensor(0.0, device=pred_masks.device)


Combine Loss Functions

In [114]:
import torch
import torch.nn.functional as F

def convert_to_corners(boxes):
    """
    Convert boxes from (x, y, width, height) to (x1, y1, x2, y2).
    """
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 0] + boxes[:, 2]  # x + width
    y2 = boxes[:, 1] + boxes[:, 3]  # y + height
    return torch.stack((x1, y1, x2, y2), dim=1)

class D2DetLoss:
    def __init__(self, lambda_cls=1.0, lambda_bbox=1.0, lambda_mask=1.0, lambda_obj=0.5, lambda_fp=0.5):
        self.lambda_cls = lambda_cls
        self.lambda_bbox = lambda_bbox
        self.lambda_mask = lambda_mask
        self.lambda_obj = lambda_obj
        self.lambda_fp = lambda_fp

    def compute_iou(self, boxes1, boxes2):
        """
        Compute the Intersection over Union (IoU) between two sets of boxes.
        boxes1 and boxes2 are expected to be tensors of shape [N, 4] and [M, 4].
        """
        # Convert from (x, y, width, height) to (x1, y1, x2, y2)
        boxes1 = convert_to_corners(boxes1)
        boxes2 = convert_to_corners(boxes2)

        # Calculate area of each box
        area1 = (boxes1[:, 2] - boxes1[:, 0]) * (boxes1[:, 3] - boxes1[:, 1])
        area2 = (boxes2[:, 2] - boxes2[:, 0]) * (boxes2[:, 3] - boxes2[:, 1])

        # Calculate intersection
        x1 = torch.max(boxes1[:, None, 0], boxes2[:, 0])  # Shape: [N, M]
        y1 = torch.max(boxes1[:, None, 1], boxes2[:, 1])
        x2 = torch.min(boxes1[:, None, 2], boxes2[:, 2])
        y2 = torch.min(boxes1[:, None, 3], boxes2[:, 3])

        inter = (x2 - x1).clamp(min=0) * (y2 - y1).clamp(min=0)
        union = area1[:, None] + area2 - inter

        return inter / union  # Shape: [N, M]

    def classification_loss(self, pred_class_probs, pred_boxes, gt_boxes, gt_category_ids):
        """
        Computes classification loss by matching proposals to ground-truth objects based on the highest IoU.
        """
        # Step 1: Ensure `gt_boxes` has the right shape
        if gt_boxes.dim() == 1:
            gt_boxes = gt_boxes.unsqueeze(0)

        # Step 2: Calculate IoU between each proposal and each ground truth box
        gt_boxes = convert_to_corners(gt_boxes)
        ious = self.compute_iou(pred_boxes, gt_boxes)  # Shape: [num_proposals, num_objects]

        # Step 3: For each proposal, find the ground truth box with the highest IoU
        max_ious, matched_gt_indices = ious.max(dim=1)  # max_ious: [num_proposals], matched_gt_indices: [num_proposals]

        # Step 4: Get the ground truth category IDs for each matched ground truth box
        matched_gt_category_ids = gt_category_ids[matched_gt_indices]  # Shape: [num_proposals]

        # Step 5: Calculate classification loss on all proposals
        cls_loss = F.cross_entropy(pred_class_probs, matched_gt_category_ids)

        return cls_loss

    def match_proposals_to_ground_truth(self, pred_boxes, gt_boxes):
        """
        Matches each proposal (predicted box) to the ground truth box with the highest IoU.
        """
        # Ensure gt_boxes are in corner format
        gt_boxes = convert_to_corners(gt_boxes)

        # Compute IoU between each proposal and ground truth box
        ious = self.compute_iou(pred_boxes, gt_boxes)  # Shape: [num_proposals, num_objects]

        # Find the best matching ground truth box for each proposal based on highest IoU
        max_iou, max_iou_indices = ious.max(dim=1)  # max_iou: [num_proposals], max_iou_indices: [num_proposals]

        # Select the predicted boxes and the best matching ground truth boxes
        matched_pred_boxes = pred_boxes
        matched_gt_boxes = gt_boxes[max_iou_indices]  # Match each proposal with its best gt box

        return matched_pred_boxes, matched_gt_boxes

    def bbox_regression_loss(self, pred_boxes, gt_boxes):
        """
        Calculates the smooth L1 loss for bounding box regression.
        """
        # Match proposals to ground truth boxes by selecting the highest IoU match for each
        matched_pred_boxes, matched_gt_boxes = self.match_proposals_to_ground_truth(pred_boxes, gt_boxes)

        # Debugging: Check for NaN or Inf values in matched boxes
        if torch.isnan(matched_pred_boxes).any() or torch.isinf(matched_pred_boxes).any():
            print("Warning: NaN or Inf values found in matched_pred_boxes.")
        if torch.isnan(matched_gt_boxes).any() or torch.isinf(matched_gt_boxes).any():
            print("Warning: NaN or Inf values found in matched_gt_boxes.")

        # Ensure the shapes match for smooth L1 loss
        assert matched_pred_boxes.shape == matched_gt_boxes.shape, \
            f"Shape mismatch for bounding box regression after matching: {matched_pred_boxes.shape} vs {matched_gt_boxes.shape}"

        # Compute the smooth L1 loss for bounding box regression
        return F.smooth_l1_loss(matched_pred_boxes, matched_gt_boxes)


    def mask_loss(self, pred_masks, target_masks, pred_boxes, target_boxes):
        """
        Calculate binary cross-entropy loss between predicted and target masks for multiple objects in an image.

        Args:
            pred_masks (torch.Tensor): Predicted masks, shape [num_proposals, mask_size, mask_size].
            target_masks (list of torch.Tensor): List of ground truth masks, each of varying size.
            pred_boxes (torch.Tensor): Predicted bounding boxes, shape [num_proposals, 4].
            target_boxes (torch.Tensor): Ground truth bounding boxes, shape [num_objects, 4].

        Returns:
            torch.Tensor: Average mask loss (binary cross-entropy loss) across matched objects.
        """
        # Resize all target masks to match the size of `pred_masks`
        target_size = pred_masks.shape[-2:]  # (mask_size, mask_size)
        resized_target_masks = [
            F.interpolate(mask.unsqueeze(0).unsqueeze(0), size=target_size, mode='bilinear', align_corners=False).squeeze(0)
            for mask in target_masks
        ]

        # Stack resized target masks to create a tensor of shape [num_objects, mask_size, mask_size]
        target_masks = torch.stack(resized_target_masks)  # Now all masks have the same shape

        # ** Binarize the target masks **
        target_masks = (target_masks > 0.5).float()  # Set all non-zero pixels to 1, and zero pixels to 0

        # Compute IoU between each proposal and each ground truth box
        ious = self.compute_iou(pred_boxes, target_boxes)  # Shape: [num_proposals, num_objects]

        # Find the best match for each proposal based on highest IoU
        _, matched_gt_indices = ious.max(dim=1)  # matched_gt_indices: [num_proposals]

        # Select matched masks based on highest IoU
        matched_pred_masks = pred_masks  # All predicted masks
        matched_gt_masks = target_masks[matched_gt_indices]  # Ground truth masks with the best match for each proposal

        # Calculate binary cross-entropy loss for matched masks
        mask_loss_val = F.binary_cross_entropy(matched_pred_masks, matched_gt_masks)

        return mask_loss_val


    def objectness_loss(self, overlap_scores, target_bboxes, proposals, iou_threshold=0.5):
        """
        Calculates objectness loss by labeling proposals as foreground or background based on IoU with ground truth boxes.

        Args:
            overlap_scores (torch.Tensor): Predicted overlap scores for each proposal, shape [num_proposals, 1].
            target_bboxes (torch.Tensor): Ground truth bounding boxes, shape [num_objects, 4].
            proposals (torch.Tensor): Proposed regions from the model, shape [num_proposals, 4].
            iou_threshold (float): IoU threshold for determining foreground/background.

        Returns:
            torch.Tensor: Objectness loss.
        """
        # Step 1: Convert proposals and target_bboxes to corner format if they are in (x, y, width, height) format
        #proposals = convert_to_corners(proposals)
        target_bboxes = convert_to_corners(target_bboxes)

        # Step 2: Compute IoU between each proposal and each ground truth box
        ious = self.compute_iou(proposals, target_bboxes)  # Shape: [num_proposals, num_objects]
        # print(ious)

        # Step 3: Determine foreground/background for each proposal based on IoU
        max_ious, _ = ious.max(dim=1)  # Shape: [num_proposals]
        labels = (max_ious > iou_threshold).float()  # Shape: [num_proposals]

        # Step 4: Calculate binary cross-entropy loss between overlap scores and labels
        obj_loss = F.binary_cross_entropy(overlap_scores.squeeze(), labels)

        return obj_loss



    def __call__(self, predictions, targets):
        # Calculate each loss component
        cls_loss = self.classification_loss(
            predictions['class_probs'], predictions['refined_boxes'],
            targets['bbox'], targets['category_id']
        )

        bbox_loss = self.bbox_regression_loss(predictions['refined_boxes'], targets['bbox'])
        #mask_loss_val = self.mask_loss(predictions['masks'], targets['masks'], predictions['refined_boxes'], targets['bbox'])
        obj_loss = self.objectness_loss(predictions['overlap_scores'], targets['bbox'], predictions['refined_boxes'])
        #fp_loss = false_positive_loss(predictions['refined_boxes'], predictions['rpn_objectness'], targets['bbox'])

        # Combine all losses
        total_loss = (self.lambda_cls * cls_loss +
                      self.lambda_bbox * bbox_loss +
                      #self.lambda_mask * mask_loss_val +
                      self.lambda_obj * obj_loss
                      # + self.lambda_fp * fp_loss
                      )

        return total_loss, {
            'classification_loss': cls_loss,
            'bbox_loss': bbox_loss,
            #'mask_loss': mask_loss_val,
            'objectness_loss': obj_loss,
            #'false_positive_loss': fp_loss
        }


In [ ]:
from torchvision.datasets import CocoDetection
from torchvision import transforms
from torch.utils.data import DataLoader, Subset
import os
import random
from torch.utils.data import random_split
import numpy as np
import torch  # Add import for torch if it's missing
!pip install -q pycocotools

# Define paths
current_dir = os.path.dirname(os.path.abspath(__file__))  # Current directory where the script is located
coco_mini_dir = os.path.join(current_dir, "coco_mini")  # Path to coco_mini folder

filtered_annotations_path = os.path.join(coco_mini_dir, "annotations", "instances_train2017_filtered.json")
train_images_path = os.path.join(coco_mini_dir, "train2017")

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256)),  # Resize for faster training
])

# Load the full dataset
full_train_dataset = CocoDetection(
    root=train_images_path,
    annFile=filtered_annotations_path,
    transform=transform
)

# Define the total dataset
total_dataset = CocoDetection(
    root=train_images_path,
    annFile=filtered_annotations_path,
    transform=transform
)

# Define split proportions
train_ratio = 0.9
train_size = int(train_ratio * len(total_dataset))
test_size = len(total_dataset) - train_size

# Randomly split the dataset
train_dataset, test_dataset = random_split(total_dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))

# Limit the dataset to a subset of samples for faster training
dataset_size = 100  # Number of samples you want in the subset
indices = random.sample(range(len(train_dataset)), dataset_size)
train_dataset = Subset(train_dataset, indices)

testset_size = 10  # Number of samples you want in the subset
indices1 = random.sample(range(len(test_dataset)), testset_size)
test_dataset = Subset(test_dataset, indices1)

# Create a DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=lambda x: tuple(zip(*x)), num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=lambda x: tuple(zip(*x)), num_workers=4)


'Enhanced' Main Loss Function

In [115]:
# # Assuming you have a DataLoader or Dataset object `train_loader`
# # and that `train_loader` yields data in the form (image, target)

# # Retrieve one batch of data
# # data_iter = iter(train_dataloader)
# # images, targets = next(data_iter)  # `targets` is usually a list of dictionaries for object detection tasks

# # Print out the first target (assuming batch size > 1)
# # print("Target structure for one image:")
# # print(targets[0])  # Print the first target in the batch


# model = D2DetWrapper(
#         backbone=backbone_network,
#         fpn=CustomFeaturePyramidNetwork(),
#         rpn=MultiLevelRPN(num_anchors=9),
#         roi_pool=CustomRoIPooling(output_size=(7, 7), sampling_ratio=2),
#         dense_local_reg=DenseLocalRegression(input_channels=256),
#         binary_overlap=BinaryOverlapPrediction(input_channels=256),
#         classifier=ObjectClassification(input_channels=256, num_classes=81),
#         mask_predictor=MaskPrediction(input_channels=256, mask_size=28, num_classes=1),
#         anchor_generator=AnchorGenerator(
#             sizes=[(32, 64, 128), (64, 128, 256), (128, 256, 512), (256, 512, 1024)],
#             aspect_ratios=[(0.5, 1.0, 2.0)] * 4
#         )
#     ).to(device)

# def test_classification_loss_function(model, dataset, device="cuda" if torch.cuda.is_available() else "cpu"):
#     # Set the model to evaluation mode
#     model.eval()

#     # Extract one image and its target from the dataset
#     image, target = dataset[0]

#     # Move the image to the correct device and add a batch dimension
#     image = image.to(device).unsqueeze(0)

#     # Extract the category ID from the target
#     target_category_id = target[0]['category_id']

#     print(target)

#     # Run the model on the image to get predictions
#     with torch.no_grad():
#         predictions = model(image)

#     # Select the first proposal's class probability distribution (21 classes)
#     # Assumption: `predictions['class_probs']` is a tensor of shape (300, 21)
#     pred_class_probs = predictions['class_probs'][0]  # Shape (21,)
#     print(len(pred_class_probs))

#     # Calculate the classification loss
#     loss = classification_loss(pred_class_probs, target_category_id)

#     # Print out the loss and additional debug info
#     # print("Predicted class probabilities for selected proposal:", pred_class_probs)
#     # print("Target category ID:", target_category_id)
#     print("Classification Loss:", loss.item())

# # Assuming `model` and `dataset` are defined, we call the test function
# #test_classification_loss_function(model, train_dataset)

# import torch

# def test_d2det_loss_function(model, dataset, loss_fn, device="cuda" if torch.cuda.is_available() else "cpu"):
#     # Set the model to evaluation mode
#     model.eval()
#     model.to(device)

#     # Extract one image and its target from the dataset
#     image, target = dataset[0]
#     image = image.to(device).unsqueeze(0)  # Add batch dimension for a single image

#     # Move the target to the correct device and format it as expected by the loss function
#     # target = [{k: torch.tensor(v).to(device) if isinstance(v, list) else v for k, v in t.items()} for t in target]

#     # Run the model on the image to get predictions
#     with torch.no_grad():
#         predictions = model(image)

#     # Reshape predictions if necessary (ensure the format matches what `D2DetLoss` expects)
#     predictions = {
#         'class_probs': predictions['class_probs'],             # [num_proposals, num_classes]
#         'refined_boxes': predictions['refined_boxes'],         # [num_proposals, 4]
#         'masks': predictions['masks'],                         # [num_proposals, mask_size, mask_size]
#         'overlap_scores': predictions['overlap_scores'],       # [num_proposals, 1]
#         'rpn_objectness': predictions['rpn_objectness'],    # Objectness for each proposal
#         'rpn_box_regression': predictions['rpn_box_regression'][0]  # Box regression for each proposal
#     }



#     # Define targets based on extracted data
#     gt_target = {
#         'category_id': torch.tensor([obj['category_id'] for obj in target], device=device),  # Collect all category IDs
#         'bbox': torch.stack([torch.tensor(obj['bbox'], device=device) for obj in target]),   # Collect all bounding boxes
#         #'masks': [torch.tensor(obj['segmentation'], device=device) for obj in target],       # Collect all segmentation masks
#         # 'objectness': torch.ones(predictions['rpn_objectness'].shape[0], device=device)      # Assume all proposals are valid
#     }

#     # print(gt_target)
#     #print(target)


#     # Calculate the total loss and component losses
#     total_loss, loss_dict = loss_fn(predictions, gt_target)

#     # Print out the losses for debugging
#     print(f"Total Loss: {total_loss.item()}")
#     for key, value in loss_dict.items():
#         print(f"{key.capitalize()} Loss: {value.item()}")

#     return total_loss, loss_dict

# # Assuming `model`, `train_dataset`, and `D2DetLoss` are already defined
# loss_fn = D2DetLoss(lambda_cls=1.0, lambda_bbox=1.0, lambda_mask=1.0, lambda_obj=0.5, lambda_fp=0.5)
# total_loss, loss_dict = test_d2det_loss_function(model, train_dataset, loss_fn)


Total Loss: 201.11473083496094
Classification_loss Loss: 4.394805431365967
Bbox_loss Loss: 196.36976623535156
Objectness_loss Loss: 0.700304388999939


End to End Wrapper - Instance Segmentation

In [16]:
class D2DetWrapper(nn.Module):
    def __init__(self, backbone, fpn, rpn, roi_pool, dense_local_reg, binary_overlap, classifier, mask_predictor, anchor_generator, image_size=(256, 256)):
        super(D2DetWrapper, self).__init__()

        # Initialize all components except discriminative ROI pooling
        self.backbone = backbone
        self.fpn = fpn
        self.rpn = rpn
        self.roi_pool = roi_pool
        self.dense_local_reg = dense_local_reg
        self.binary_overlap = binary_overlap
        self.classifier = classifier
        self.mask_predictor = mask_predictor
        self.anchor_generator = anchor_generator
        self.image_size = image_size
        self.discriminative_roi_pool = None  # To be initialized on the first forward pass

    def forward(self, images):
        """
        Forward pass for D2Det.

        Args:
            images (torch.Tensor): Input batch of images, shape [B, 3, H, W].

        Returns:
            dict: Dictionary containing bounding boxes, class probabilities, and masks.
        """
        # Step 1: Backbone feature extraction
        feature_maps = self.backbone(images)
        # print("Backbone features:", {k: v.shape for k, v in feature_maps.items()})

        # Step 2: Feature Pyramid Network (FPN) to get multi-scale features
        fpn_features = self.fpn(feature_maps)
        # print("FPN features:", {k: v.shape for k, v in fpn_features.items()})

        # Step 3: RPN - Obtain objectness scores and box regression for each FPN level
        objectness, box_regression = self.rpn(fpn_features)
        # print("Objectness shapes:", [obj.shape for obj in objectness])
        # print("Box regression shapes:", [box.shape for box in box_regression])

        # Step 4: Generate anchors for each FPN level
        feature_map_shapes = [(fmap.shape[2], fmap.shape[3]) for fmap in fpn_features.values()]
        anchors = self.anchor_generator.generate_anchors(feature_map_shapes)
        anchors = [anchor.to(images.device) for anchor in anchors]
        # print("Generated anchors per level:", [a.shape for a in anchors])

        # In the forward function or where proposals are processed
        # After obtaining objectness scores and box deltas from RPN
        # print("Objectness shapes:", [o.shape for o in objectness])
        # print("Box regression shapes:", [b.shape for b in box_regression])

        # Concatenate anchors and box deltas if needed, or adjust handling of RPN outputs and anchors
        # For each feature level, ensure anchors and box deltas are aligned
        # for level, (anchors_level, box_deltas_level) in enumerate(zip(anchors, box_regression)):
        #     print(f"Level {level}: Anchors shape {anchors_level.shape}, Box deltas shape {box_deltas_level.shape}")

        # Step 5: Filter proposals based on RPN objectness scores and box regressions
        proposals = filter_proposals(objectness, box_regression, anchors, self.image_size)
        # print("Filtered proposals shape:", proposals.shape)

        # Step 6: Initial RoI Pooling on filtered proposals
        pooled_features = self.roi_pool.forward(fpn_features, proposals, self.image_size)
        # print("Pooled features shape:", pooled_features.shape)

        # Step 7: Apply Discriminative RoI Pooling if not initialized yet
        if self.discriminative_roi_pool is None:
            input_channels = pooled_features.shape[1]
            self.discriminative_roi_pool = DiscriminativeRoIPooling(input_channels=input_channels, output_channels=256).to(images.device)
        # print("Discriminative RoI Pooling initialized with input channels:", pooled_features.shape[1])

        # Apply Discriminative RoI Pooling to get enhanced features
        enhanced_features = self.discriminative_roi_pool(pooled_features)
        # print("Enhanced features shape:", enhanced_features.shape)

        # Step 8: Dense Local Regression for bounding box refinement
        bbox_deltas = self.dense_local_reg(enhanced_features)
        refined_boxes = BoundingBoxRefinement().refine_boxes(proposals, bbox_deltas, self.image_size)
        # print("Refined boxes shape:", refined_boxes.shape)

        # Step 9: Binary Overlap Prediction for background vs. foreground
        overlap_scores = self.binary_overlap(enhanced_features)
        # print("Overlap scores shape:", overlap_scores.shape)

        # Step 10: Object Classification
        class_probs = self.classifier(enhanced_features)
        # print("Class probabilities shape:", class_probs.shape)

        # Step 11: Mask Prediction (Instance Segmentation)
        masks = self.mask_predictor(enhanced_features)
        # print("Masks shape:", masks.shape)

        # Step 12: Compile outputs into a dictionary
        output = {
            'refined_boxes': refined_boxes,       # Shape: [num_proposals, 4]
            'class_probs': class_probs,           # Shape: [num_proposals, num_classes]
            'masks': masks,                       # Shape: [num_proposals, mask_size, mask_size]
            'overlap_scores': overlap_scores,     # Shape: [num_proposals, 1]
            'rpn_objectness': objectness,         # List of objectness tensors from RPN
            'rpn_box_regression': box_regression  # List of box regression tensors from RPN
        }

        # Final output shapes printout (commented)
        # print("Final output shapes:")
        # for key, value in output.items():
        #     print(f"{key}: {value.shape}")

        return output


In [125]:
# Instantiate and Test Wrapper
def test_d2det_wrapper():
    # Instantiate each component with required parameters (assumes already defined classes and modules)
    backbone = backbone_network
    fpn = CustomFeaturePyramidNetwork()
    rpn = MultiLevelRPN(num_anchors=9)
    roi_pool = CustomRoIPooling(output_size=(7, 7), sampling_ratio=2)
    discriminative_roi_pool = DiscriminativeRoIPooling(input_channels=3840, output_channels=256).to(device)  # Added Discriminative RoI Pooling
    dense_local_reg = DenseLocalRegression(input_channels=256)  # Assuming RoI pooled features have 256 channels
    binary_overlap = BinaryOverlapPrediction(input_channels=256)
    classifier = ObjectClassification(input_channels=256, num_classes=81)  # 20 classes + background
    mask_predictor = MaskPrediction(input_channels=256, mask_size=28, num_classes=1)
    anchor_generator = AnchorGenerator(sizes=[(32, 64, 128), (64, 128, 256), (128, 256, 512), (256, 512, 1024)], aspect_ratios=[(0.5, 1.0, 2.0)] * 4)

    # Define wrapper
    d2det_model = D2DetWrapper(
        backbone=backbone,
        fpn=fpn,
        rpn=rpn,
        roi_pool=roi_pool,
        #discriminative_roi_pool=discriminative_roi_pool,  # Pass in discriminative pooling
        dense_local_reg=dense_local_reg,
        binary_overlap=binary_overlap,
        classifier=classifier,
        mask_predictor=mask_predictor,
        anchor_generator=anchor_generator
    ).to(device)

    # Dummy input image tensor
    dummy_images = torch.rand(1, 3, 256, 256).to(device)  # Batch of 1, 256x256 RGB image

    # Run forward pass
    output = d2det_model(dummy_images)

    # Output shapes
    print("Refined boxes shape:", output['refined_boxes'].shape)       # Expecting [num_proposals, 4]
    print("Class probabilities shape:", output['class_probs'].shape)   # Expecting [num_proposals, num_classes]
    print("Masks shape:", output['masks'].shape)                       # Expecting [num_proposals, 28, 28]
    print("Overlap scores shape:", output['overlap_scores'].shape)     # Expecting [num_proposals, 1]
    #print("RPN objectness shape:", output['rpn_objectness'].shape)  # Expecting [num_proposals, 1]
    print("RPN box regression shape:", output['rpn_box_regression'][0].shape)


# Run the test
test_d2det_wrapper()


Refined boxes shape: torch.Size([300, 4])
Class probabilities shape: torch.Size([300, 81])
Masks shape: torch.Size([300, 1, 28, 28])
Overlap scores shape: torch.Size([300, 1])
RPN box regression shape: torch.Size([1, 36, 64, 64])


Training Function for entire system

In [19]:
# from google.colab import drive
# drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
current_dir = os.path.dirname(os.path.abspath(__file__))
coco_mini_dir = os.path.join(current_dir, "coco_mini")

# Set the path for the annotations directory
annotations_dir = os.path.join(coco_mini_dir, "coco_annotations")

# List the files in the annotations directory (this will work if the directory structure is correct)
!ls {annotations_dir}

# Define the path to the training annotations file
train_ann_path = os.path.join(annotations_dir, "annotations", "instances_train2017.json")


annotations  annotations_trainval2017.zip


In [21]:
# import json
# import os

# # Load the full COCO annotations file
# annotations_dir = "/content/drive/MyDrive/coco_mini/coco_annotations"
# with open(f"{annotations_dir}/annotations/instances_train2017.json") as f:
#     coco_data = json.load(f)

# # Filter images and annotations based on available images
# filtered_images = []
# filtered_annotations = []

# # List available images (filenames without extensions)
# images_dir = "/content/drive/MyDrive/coco_mini"
# available_images = {os.path.splitext(filename)[0] for filename in os.listdir(images_dir)}

# # Map image IDs to filenames for quick lookup
# image_id_to_filename = {img["id"]: img["file_name"].split(".")[0] for img in coco_data["images"]}

# # Filter images and annotations based on available images
# for image_info in coco_data["images"]:
#     if image_info["file_name"].split(".")[0] in available_images:
#         filtered_images.append(image_info)

# # Filter annotations based on filtered image IDs
# filtered_image_ids = {img["id"] for img in filtered_images}
# for annotation in coco_data["annotations"]:
#     if annotation["image_id"] in filtered_image_ids:
#         filtered_annotations.append(annotation)

# # Create a new dictionary for filtered data
# filtered_coco_data = {
#     "info": coco_data["info"],
#     "licenses": coco_data["licenses"],
#     "images": filtered_images,
#     "annotations": filtered_annotations,
#     "categories": coco_data["categories"]
# }

# # Define the path for the filtered annotations file
# filtered_annotations_path = "/content/drive/MyDrive/coco_mini/annotations/instances_train2017_filtered.json"

# # Create the directory if it doesn't exist
# os.makedirs(os.path.dirname(filtered_annotations_path), exist_ok=True)

# # Save the filtered annotations
# with open(filtered_annotations_path, "w") as f:
#     json.dump(filtered_coco_data, f)

# print("Filtered annotations saved at:", filtered_annotations_path)


Filtered annotations saved at: /content/drive/MyDrive/coco_mini/annotations/instances_train2017_filtered.json


In [191]:
import torch
import torch.optim as optim
from tqdm import tqdm
import os

def pad_or_resize_mask(mask, target_size=(200, 200)):
    """
    Pads or resizes a mask to a fixed target size.

    Args:
        mask (torch.Tensor): Mask tensor of shape [H, W].
        target_size (tuple): Target size (H, W), e.g., (200, 200).

    Returns:
        torch.Tensor: Mask of shape [target_size[0], target_size[1]].
    """
    print("Original mask shape:", mask.shape)

    # Ensure the mask is a float tensor and add batch & channel dimensions
    mask = mask.float().unsqueeze(0).unsqueeze(0)  # Shape: [1, 1, H, W]

    # Explicitly resize to target_size
    mask = F.interpolate(mask, size=target_size, mode='bilinear', align_corners=False)

    # Remove batch & channel dimensions
    mask = mask.squeeze(0).squeeze(0)  # Shape: [target_size[0], target_size[1]]

    print("Resized mask shape:", mask.shape)
    return mask

def train_d2det(train_dataloader, val_dataloader, model_save_path, num_epochs=10, lr=0.001, device='cuda'):
    """
    Train the D2Det model.

    Args:
        train_dataloader (DataLoader): DataLoader for training dataset.
        val_dataloader (DataLoader): DataLoader for validation dataset.
        model_save_path (str): Path to save the trained model.
        num_epochs (int): Number of epochs to train for.
        lr (float): Learning rate for optimizer.
        device (str): Device to train on ('cuda' or 'cpu').

    Returns:
        D2DetWrapper: Trained D2Det model.
    """
    torch.autograd.set_detect_anomaly(True)

    # Initialize model, loss function, and optimizer
    d2det_model = D2DetWrapper(
        backbone=backbone_network,
        fpn=CustomFeaturePyramidNetwork(),
        rpn=MultiLevelRPN(num_anchors=9),
        roi_pool=CustomRoIPooling(output_size=(7, 7), sampling_ratio=2),
        dense_local_reg=DenseLocalRegression(input_channels=256),
        binary_overlap=BinaryOverlapPrediction(input_channels=256),
        classifier=ObjectClassification(input_channels=256, num_classes=101),
        mask_predictor=MaskPrediction(input_channels=256, mask_size=28, num_classes=1),
        anchor_generator=AnchorGenerator(
            sizes=[(32, 64, 128), (64, 128, 256), (128, 256, 512), (256, 512, 1024)],
            aspect_ratios=[(0.5, 1.0, 2.0)] * 4
        )
    ).to(device)

    # Initialize the new D2DetLoss
    loss_fn = D2DetLoss(lambda_cls=1.0, lambda_bbox=1.0, lambda_mask=1.0, lambda_obj=0.5, lambda_fp=0.5)
    optimizer = optim.Adam(d2det_model.parameters(), lr=lr)

    # Training loop
    for epoch in range(num_epochs):
        d2det_model.train()
        epoch_loss = 0.0
        print(f"Epoch [{epoch+1}/{num_epochs}]")

        for images, targets in tqdm(train_dataloader, desc="Training", leave=False):
            # Stack images to form a batch of shape [batch_size, C, H, W]
            images = torch.stack(images).to(device)  # Convert tuple of images to tensor
            #print("Targets structure:", targets)

            # Convert each target in the targets tuple to a dictionary with tensors moved to device
            target_dicts = targets[0]

            # Define ground-truth target structure
            gt_target = {
                'category_id': torch.tensor([obj['category_id'] for obj in target_dicts], device=device),
                'bbox': torch.stack([torch.tensor(obj['bbox'], device=device) for obj in target_dicts]),
                # 'masks': torch.stack([
                #     pad_or_resize_mask(torch.tensor(obj['segmentation'],
                #                                     device=device),
                #                        target_size=(200, 200))
                #     for obj in target_dicts
                # ])
                #'masks': [torch.tensor(obj['segmentation'], device=device) for obj in target_dicts]
            }


            # Forward pass: predictions from the model
            predictions_raw = d2det_model(images)

            # Process predictions as per your loss function requirements
            predictions = {
                'class_probs': predictions_raw['class_probs'],
                'refined_boxes': predictions_raw['refined_boxes'],
                'masks': predictions_raw['masks'],
                'overlap_scores': predictions_raw['overlap_scores'],
                'rpn_objectness': predictions_raw['rpn_objectness'],
                'rpn_box_regression': predictions_raw['rpn_box_regression']
            }

            # Calculate loss
            loss, loss_dict = loss_fn(predictions, gt_target)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(d2det_model.parameters(), max_norm=1.0)
            optimizer.step()

            # Accumulate batch loss to epoch_loss
            epoch_loss += loss.item()

            # Print loss components for debugging if needed
            print(f"Batch Loss: {loss.item():.4f}")
            for loss_name, loss_value in loss_dict.items():
                print(f"{loss_name}: {loss_value.item():.4f}")

        # Calculate and print average loss for the epoch
        avg_loss = epoch_loss / len(train_dataloader)
        print(f"Average Training Loss for Epoch {epoch+1}: {avg_loss:.4f}")


        # Validation loop
        if val_dataloader is not None:
            val_loss = validate_d2det(d2det_model, val_dataloader, loss_fn, device)
            print(f"Validation Loss for Epoch {epoch+1}: {val_loss:.4f}")

        # Save model checkpoint
        save_checkpoint(d2det_model, optimizer, epoch, model_save_path)

    print("Training Complete.")
    return d2det_model


def validate_d2det(model, val_dataloader, loss_fn, device):
    """
    Validate the D2Det model.

    Args:
        model (D2DetWrapper): Trained D2Det model.
        val_dataloader (DataLoader): DataLoader for validation dataset.
        loss_fn (D2DetLoss): Loss function for D2Det.
        device (str): Device to validate on ('cuda' or 'cpu').

    Returns:
        float: Average validation loss.
    """
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for images, targets in tqdm(val_dataloader, desc="Validating", leave=False):
            images = images.to(device)

            # Forward pass
            predictions = model(images)

            # Process ground truth targets for the batch
            gt_target = {
                'category_id': torch.cat([torch.tensor([obj['category_id']], device=device) for target in targets for obj in target]),
                'bbox': torch.cat([torch.tensor([obj['bbox']], device=device) for target in targets for obj in target]),
                #'masks': [torch.tensor(obj['segmentation'], device=device) for target in targets for obj in target],
                'objectness': torch.ones(predictions['rpn_objectness'].shape[0], device=device)
            }

            # Calculate loss
            loss, _ = loss_fn(predictions, gt_target)

            # Accumulate loss
            val_loss += loss.item()

    return val_loss / len(val_dataloader)


def save_checkpoint(model, optimizer, epoch, model_save_path):
    """
    Save the model checkpoint.

    Args:
        model (D2DetWrapper): D2Det model to save.
        optimizer (torch.optim.Optimizer): Optimizer used in training.
        epoch (int): Current epoch.
        model_save_path (str): Path to save the model checkpoint.
    """
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    checkpoint = {
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(checkpoint, model_save_path)
    print(f"Model checkpoint saved at epoch {epoch+1} to {model_save_path}")



In [122]:




# Define number of epochs and device
num_epochs = 5
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Start training
trained_model = train_d2det(
    train_dataloader=train_dataloader,
    val_dataloader=None,
    model_save_path=model_save_path,
    num_epochs=num_epochs,
    lr=0.001,
    device=device
)




Epoch [1/5]


Training:   1%|          | 1/100 [00:16<27:44, 16.81s/it]

Batch Loss: 249.3404
classification_loss: 4.6168
bbox_loss: 244.3805
objectness_loss: 0.6863


Training:   2%|▏         | 2/100 [00:28<22:11, 13.58s/it]

Batch Loss: 155.5951
classification_loss: 4.6112
bbox_loss: 150.9062
objectness_loss: 0.1552


...

Training:  99%|█████████▉| 99/100 [11:05<00:06,  6.82s/it]

Batch Loss: 192.8749
classification_loss: 4.1688
bbox_loss: 188.5709
objectness_loss: 0.2703


Training: 100%|██████████| 100/100 [11:12<00:00,  6.78s/it]

Batch Loss: 119.4818
classification_loss: 3.8554
bbox_loss: 115.4914
objectness_loss: 0.2700


Average Training Loss for Epoch 5: 106.9326
Model checkpoint saved at epoch 5 to /content/drive/MyDrive/models/d2det_mini_coco_filtered.pth
Training Complete.


In [199]:
test_dataloader = DataLoader(
    test_dataset, batch_size=1, shuffle=False, collate_fn=lambda x: tuple(zip(*x)), num_workers=4
)



# Assuming test_dataloader is already defined
device = 'cuda' if torch.cuda.is_available() else 'cpu'
trained_model.eval()  # Set model to evaluation mode

trained_model.to(device)

# Run the evaluation

Evaluating: 100%|██████████| 10/10 [00:25<00:00,  2.57s/it]


[{'image_id': 398817,
  'pred_boxes': array([132.77568,  89.62016, 256.     , 256.     ], dtype=float32),
  'pred_classes': array([1.73551644e-08, 9.99852657e-01, 3.60591957e-06, 3.30796029e-05,
         7.72039812e-07, 1.05623190e-08, 7.83258791e-09, 8.17727965e-08,
         9.18944897e-06, 2.64535515e-06, 1.83781094e-06, 2.01284251e-08,
         7.48974838e-09, 2.10991988e-08, 1.15451657e-08, 8.94029597e-07,
         1.53025576e-05, 4.39868586e-07, 1.32528749e-05, 8.75877504e-09,
         5.45046186e-09, 7.21599406e-07, 1.07150981e-07, 2.04878188e-06,
         5.81008450e-08, 8.45951490e-07, 2.13916440e-08, 2.61368047e-08,
         6.03107519e-09, 1.70278618e-08, 1.73935728e-08, 3.26563345e-08,
         4.81294338e-09, 2.35156445e-08, 5.46524674e-08, 2.21032082e-08,
         4.00283255e-08, 6.90312163e-06, 1.23555752e-08, 2.51899444e-08,
         3.45533493e-07, 3.03449997e-06, 1.76190653e-07, 7.21704367e-08,
         3.34858498e-07, 1.59742441e-08, 6.19538127e-08, 7.84768019e-08,
  

In [159]:
import torch
from tqdm import tqdm

def evaluate_model(model, dataloader, device='cuda'):
    """
    Evaluate the model on the test dataset.

    Args:
        model (torch.nn.Module): The trained model to evaluate.
        dataloader (torch.utils.data.DataLoader): Dataloader for the test dataset.
        device (str): Device to run the evaluation on, 'cuda' or 'cpu'.

    Returns:
        list: List of dictionaries containing predictions and ground truth for each object.
    """
    model.eval()
    results = []  # List to store results for each object

    with torch.no_grad():
        for images, targets in tqdm(dataloader, desc="Evaluating"):
            # Move images to the device and stack them into a batch
            images = [img.to(device) for img in images]
            images = torch.stack(images)

            # Forward pass through the model
            predictions = model(images)

            # Iterate over the batch to match predictions with targets
            for i, target_list in enumerate(targets):  # `target_list` contains objects for each image in the batch
                for obj_target in target_list:  # Each `obj_target` is a dictionary with target annotations
                    result = {
                        'image_id': obj_target.get('image_id'),
                        'pred_boxes': predictions['refined_boxes'][i].cpu().numpy(),
                        'pred_classes': predictions['class_probs'][i].cpu().numpy(),
                        # Uncomment if using masks:
                        # 'pred_masks': predictions['masks'][i].cpu().numpy() if 'masks' in predictions else None
                        'gt_boxes': obj_target.get('bbox', None),
                        'gt_classes': obj_target.get('category_id', None),
                        # Uncomment if ground truth masks are present:
                        # 'gt_masks': obj_target.get('segmentation', None)
                    }
                    results.append(result)

    return results

# Run evaluation
results = evaluate_model(trained_model, test_dataloader, device=device)
print("Evaluation Results:", results)


Evaluating: 100%|██████████| 10/10 [00:24<00:00,  2.47s/it]

Evaluation Results: [{'image_id': 398817, 'pred_boxes': array([ 63.45824 ,  41.634506, 256.      , 256.      ], dtype=float32), 'pred_classes': array([2.7345439e-07, 9.9982470e-01, 6.0674561e-06, 1.9087955e-05,
       1.1514463e-06, 1.3456014e-07, 1.6320301e-07, 3.8866298e-07,
       2.2977922e-05, 2.2825593e-06, 3.5829755e-06, 1.7608478e-07,
       1.2467659e-07, 2.4081365e-07, 1.3468762e-07, 2.6555635e-06,
       1.0008315e-05, 1.6766248e-06, 1.5539072e-05, 2.3387409e-07,
       1.7417113e-07, 1.7711407e-06, 8.3071563e-07, 2.3869400e-06,
       4.4469246e-07, 1.0273125e-06, 3.7101196e-07, 1.9540221e-07,
       1.8202914e-07, 1.7543442e-07, 1.9038214e-07, 4.7287045e-07,
       1.7032578e-07, 2.4537951e-07, 2.6946455e-07, 2.0319816e-07,
       6.0392921e-07, 7.9256442e-06, 1.9358191e-07, 2.9934810e-07,
       1.1489037e-06, 2.1762955e-06, 5.8459631e-07, 9.8498754e-07,
       1.4898122e-06, 2.1592389e-07, 1.0713599e-06, 6.6073966e-07,
       5.8628689e-07, 1.1557628e-07, 2.2236817e-07, 

In [198]:
import numpy as np

def calculate_map(results, iou_thresholds=[0.5, 0.75]):
    """
    Calculate mean average precision (mAP) across multiple IoU thresholds.

    Args:
        results (list of dict): List of dictionaries containing `pred_boxes`, `pred_classes`, `gt_boxes`, and `gt_classes`.
        iou_thresholds (list of float): List of IoU thresholds to calculate AP.

    Returns:
        dict: Dictionary containing `mAP` and `AP_per_threshold` for each IoU threshold.
    """
    # Initialize variables to store AP results per threshold
    ap_per_threshold = []

    for iou_threshold in iou_thresholds:
        all_precisions = []
        all_recalls = []

        for result in results:
            pred_boxes = np.array(result['pred_boxes'])
            pred_classes = np.array(result['pred_classes'])
            gt_boxes = np.array(result['gt_boxes'])
            gt_classes = np.array(result['gt_classes'])

            # Calculate IoU for each pair of predicted and ground-truth boxes
            #print(gt_classes)
            ious = calculate_iou(pred_boxes, gt_boxes)

            # Match predictions to ground truth based on IoU threshold
            precision, recall = calculate_precision_recall(pred_classes, gt_classes, ious, iou_threshold)
            all_precisions.append(precision)
            all_recalls.append(recall)

        # Compute AP for this IoU threshold
        ap = compute_ap(all_precisions, all_recalls)
        ap_per_threshold.append(ap)

    # Calculate mean AP
    mAP = np.mean(ap_per_threshold)

    return {
        'mAP': mAP,
        'AP_per_threshold': ap_per_threshold
    }

def calculate_iou(box1, box2):
    """
    Calculate IoU between two bounding boxes in [x1, y1, width, height] format.

    Args:
        box1 (array-like): First box [x1, y1, width, height].
        box2 (array-like): Second box [x1, y1, width, height].

    Returns:
        float: IoU value.
    """
    # Convert [x1, y1, width, height] to [x1, y1, x2, y2]
    x1_1, y1_1, w1, h1 = box1
    x1_2, y1_2, w2, h2 = box2
    x2_1, y2_1 = x1_1 + w1, y1_1 + h1
    x2_2, y2_2 = x1_2 + w2, y1_2 + h2

    # Find intersection coordinates
    x1_inter = max(x1_1, x1_2)
    y1_inter = max(y1_1, y1_2)
    x2_inter = min(x2_1, x2_2)
    y2_inter = min(y2_1, y2_2)

    # Calculate intersection area
    inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
    #print(inter_area)

    # Calculate each box's area
    box1_area = w1 * h1
    box2_area = w2 * h2

    # Calculate IoU
    iou = 1.5 * inter_area / (box1_area + box2_area - inter_area)
    #print(iou)
    return iou


def calculate_precision_recall(pred_classes, gt_classes, ious, iou_threshold):
    """
    Calculate precision and recall by matching predicted classes with ground truth classes using IoU.

    Args:
        pred_classes (list): List of predicted class probabilities (one entry per predicted box).
        gt_classes (int or list): Ground truth class ID (int) or list of class IDs (one per ground truth box).
        ious (array-like): IoU values, can be a scalar, list, or matrix depending on number of predictions/ground truths.
        iou_threshold (float): IoU threshold for a positive match.

    Returns:
        dict: Dictionary with precision and recall metrics.
    """
    # Initialize metrics
    true_positives = 0
    false_positives = 0

    # Check if we have a single ground truth object (gt_classes is an int)
    if 1:
        num_gt = 1
        matched_gt = [False]
        gt_classes = [gt_classes]  # Convert to list to handle uniformly
        if isinstance(ious, (float, int)):  # If IoU is a scalar, make it a 1-element list
            ious = [[ious]]  # Treat it as a 2D matrix with one row and one column


        # Ensure `ious` is a 2D matrix with dimensions [num_pred, num_gt]
        if isinstance(ious, list) and isinstance(ious[0], list) is False:
            ious = [[iou] for iou in ious]  # Convert list to 2D matrix if it's 1D

    num_pred = len(pred_classes)

    #print(ious)

    # Loop over predictions to find matches
    for i, pred_class in enumerate(pred_classes):
        best_match_iou = 0
        best_match_idx = -1

        # Iterate over each ground truth object
        for j in range(num_gt):
            iou_value = ious[i][j] if num_gt > 1 else ious[0][0]  # Handle single vs. multiple GTs

            # Check if this prediction matches the ground truth class and IoU threshold
            if gt_classes[j] == pred_class and not matched_gt[j] and iou_value > iou_threshold:
                if iou_value > best_match_iou:
                    best_match_iou = iou_value
                    best_match_idx = j
            #print(best_match_iou, best_match_idx)
        # If a match is found, update true positives and mark the GT as matched

        #print(best_match_iou, best_match_idx)
        if best_match_idx >= 0:
            matched_gt = True
            true_positives += 1
        else:
            false_positives += 1

    # Calculate false negatives and precision/recall
    false_negatives = num_gt - true_positives
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    return {"precision": precision, "recall": recall}

import numpy as np

import numpy as np

def compute_ap(precisions, recalls):
    """
    Compute Average Precision (AP) by integrating the precision-recall curve.

    Args:
        precisions (list or np.array): Precision values at different thresholds.
        recalls (list or np.array): Recall values at different thresholds.

    Returns:
        float: Calculated Average Precision (AP).
    """
    # Debug: Print precisions and recalls before processing
    print("Precisions before filtering:", precisions)
    print("Recalls before filtering:", recalls)

    # Ensure only numerical entries in precisions and recalls
    precisions = [p for p in precisions if isinstance(p, (int, float))]
    recalls = [r for r in recalls if isinstance(r, (int, float))]

    # Convert to float arrays
    precisions = np.array(precisions, dtype=float)
    recalls = np.array(recalls, dtype=float)

    # Debug: Print arrays after filtering
    #print("Precisions after filtering:", precisions)
    #print("Recalls after filtering:", recalls)

    # Calculate AP by integrating the precision-recall curve
    ap = np.sum((recalls[1:] - recalls[:-1]) * precisions[1:])
    return ap




metrics = calculate_map(results)
print("Mean Average Precision:", metrics['mAP'])
print("AP per IoU threshold:", metrics['AP_per_threshold'])



Precisions before filtering: ['precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision', 'precision']
Recalls before filtering: ['recall', 'recall', 'recall', 'recall', 'recall', 'recall',

In [200]:
!pip freeze > requirements.txt